In [1]:
# Import all the necessary modules
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
import pandas_datareader as pdr
import math
import datetime
import itertools
import ast
import yfinance as yf
import seaborn as sn
from IPython.display import display, HTML
from trend_following_signal import (apply_jupyter_fullscreen_css, load_financial_data, get_returns_volatility, calculate_slope, trend_signal, slope_signal, 
                             create_trend_strategy, get_close_prices, calculate_donchian_channels)
from strategy_performance import (calculate_sharpe_ratio, calculate_calmar_ratio, calculate_CAGR, calculate_risk_and_performance_metrics,
                                          calculate_compounded_cumulative_returns, estimate_fee_per_trade, rolling_sharpe_ratio)
import coinbase_utils as cn
import strategy_performance as perf
import position_sizing_binary_utils as size_bin
import position_sizing_continuous_utils as size_cont
import trend_following_signal as tf
%matplotlib inline

In [3]:
import importlib
importlib.reload(cn)
importlib.reload(perf)
importlib.reload(tf)
importlib.reload(size_bin)
importlib.reload(size_cont)

<module 'position_sizing_continuous_utils' from '/Users/adheerchauhan/Documents/git/trend_following/position_sizing_continuous_utils.py'>

In [5]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
apply_jupyter_fullscreen_css()

## Helper Functions

In [8]:
from collections import OrderedDict

def print_strategy_params():
    """
    Pretty-print the strategy’s configuration values, with a blank line
    separating each logical section.
    """

    # ---- Define sections (title is just for dev readability) --------------
    sections = [
        ("Dates & universe", OrderedDict([
            ("start_date",      start_date),
            ("end_date",        end_date),
            ("warm_up_days",    WARMUP_DAYS),
            ("ticker_list",     ticker_list),
        ])),

        ("Moving-average / trend", OrderedDict([
            ("fast_mavg",                  fast_mavg),
            ("slow_mavg",                  slow_mavg),
            ("mavg_stepsize",              mavg_stepsize),
            ("mavg_z_score_window",        mavg_z_score_window),
            ("moving_avg_type",            moving_avg_type),
            ("ma_crossover_signal_weight", ma_crossover_signal_weight),
        ])),

        ("Donchian channel", OrderedDict([
            ("entry_rolling_donchian_window", entry_rolling_donchian_window),
            ("exit_rolling_donchian_window", exit_rolling_donchian_window),
            ("use_donchian_exit_gate", use_donchian_exit_gate),
            ("donchian_signal_weight",  donchian_signal_weight),
        ])),

        ("Volatility & risk", OrderedDict([
            ("volatility_window",            volatility_window),
            ("annualized_target_volatility", annualized_target_volatility),
            ("rolling_cov_window",           rolling_cov_window),
            ("rolling_atr_window",           rolling_atr_window),
            ("atr_multiplier",               atr_multiplier),
            ("log_std_window",               log_std_window),
            ("coef_of_variation_window",     coef_of_variation_window),
            ("vol_of_vol_z_score_window",    vol_of_vol_z_score_window),
            ("vol_of_vol_p_min",             vol_of_vol_p_min),
            ("r2_strong_threshold",          r2_strong_threshold)
        ])),

        ("Signal gating / quality", OrderedDict([
            ("lower_r_sqr_limit",             lower_r_sqr_limit),
            ("upper_r_sqr_limit",             upper_r_sqr_limit),
            ("rolling_r2_window",             rolling_r2_window),
            ("r2_smooth_window",              r2_smooth_window),
            ("r2_confirm_days",               r2_confirm_days),
            ("rolling_sharpe_window",         rolling_sharpe_window),
            ("use_activation",                use_activation),
            ("tanh_activation_constant_dict", tanh_activation_constant_dict),
            ("weighted_signal_ewm_window",    weighted_signal_ewm_window)
        ])),

        ("Trading toggles & thresholds", OrderedDict([
            ("long_only",                  long_only),
            ("use_coinbase_data",          use_coinbase_data),
            ("use_saved_files",            use_saved_files),
            ("saved_file_end_date",        saved_file_end_date),
            ("use_specific_start_date",    use_specific_start_date),
            ("signal_start_date",          signal_start_date),
            ("price_or_returns_calc",      price_or_returns_calc),
            ("notional_threshold_pct",     notional_threshold_pct),
            ("cooldown_counter_threshold", cooldown_counter_threshold),
            ("warmup_days",                WARMUP_DAYS)
        ])),

        ("Capital & execution", OrderedDict([
            ("initial_capital",        initial_capital),
            ("cash_buffer_percentage", cash_buffer_percentage),
            ("transaction_cost_est",   transaction_cost_est),
            ("passive_trade_rate",     passive_trade_rate),
            ("annual_trading_days",    annual_trading_days),
        ])),
    ]

    # ---- Compute width for neat alignment ---------------------------------
    longest_key = max(len(k) for _, sec in sections for k in sec)

    print("\nStrategy Parameters\n" + "-" * (longest_key + 30))
    for _, sec in sections:
        for k, v in sec.items():
            print(f"{k:<{longest_key}} : {v}")
        print()  # blank line between sections
    print("-" * (longest_key + 30) + "\n")

# ---------------------------------------------------------------------------
# Example usage (uncomment after your own parameter definitions are in scope)
# ---------------------------------------------------------------------------
# if __name__ == "__main__":
#     print_strategy_params()

In [10]:
def plot_signal_performance(df_1, df_2, ticker):

    fig = plt.figure(figsize=(20,12))
    layout = (2,2)
    signal_ax = plt.subplot2grid(layout, (0,0))
    price_ax = signal_ax.twinx()
    equity_curve_ax = plt.subplot2grid(layout, (0,1))
    sharpe_ax = plt.subplot2grid(layout, (1,0))
    portfolio_value_ax = plt.subplot2grid(layout, (1,1))

    _ = signal_ax.plot(df_1.index, df_1[f'{ticker}_final_signal'], label='Orig Signal', alpha=0.9)
    _ = signal_ax.plot(df_2.index, df_2[f'{ticker}_final_signal'], label='New Signal', alpha=0.9)
    _ = price_ax.plot(df_1.index, df_2[f'{ticker}_open'], label='Price', alpha=0.7, linestyle='--', color='magenta')
    _ = signal_ax.set_title(f'Orignal Signal vs New Signal')
    _ = signal_ax.set_ylabel('Signal')
    _ = signal_ax.set_xlabel('Date')
    _ = signal_ax.legend(loc='upper left')
    _ = signal_ax.grid()

    _ = equity_curve_ax.plot(df_1.index, df_1[f'equity_curve'], label='Orig Signal', alpha=0.9)
    _ = equity_curve_ax.plot(df_2.index, df_2[f'equity_curve'], label='New Signal', alpha=0.9)
    _ = equity_curve_ax.set_title(f'Equity Curve')
    _ = equity_curve_ax.set_ylabel('Equity Curve')
    _ = equity_curve_ax.set_xlabel('Date')
    _ = equity_curve_ax.legend(loc='upper left')
    _ = equity_curve_ax.grid()

    _ = sharpe_ax.plot(df_1.index, df_1[f'portfolio_rolling_sharpe_50'], label='Orig Signal', alpha=0.9)
    _ = sharpe_ax.plot(df_2.index, df_2[f'portfolio_rolling_sharpe_50'], label='New Signal', alpha=0.9)
    _ = sharpe_ax.set_title(f'Rolling Sharpe')
    _ = sharpe_ax.set_ylabel(f'Rolling Sharpe')
    _ = sharpe_ax.set_xlabel('Date')
    _ = sharpe_ax.legend(loc='upper left')
    _ = sharpe_ax.grid()

    _ = portfolio_value_ax.plot(df_1.index, df_1[f'total_portfolio_value'], label='Orig Signal', alpha=0.9)
    _ = portfolio_value_ax.plot(df_2.index, df_2[f'total_portfolio_value'], label='New Signal', alpha=0.9)
    _ = portfolio_value_ax.set_title(f'Total Portfolio Value')
    _ = portfolio_value_ax.set_ylabel('Portfolio Value')
    _ = portfolio_value_ax.set_xlabel('Date')
    _ = portfolio_value_ax.legend(loc='upper left')
    _ = portfolio_value_ax.grid()

    plt.tight_layout()

    return

## Walk Forward Analysis for Target Volatility

In [13]:
start_date = pd.to_datetime('2022-04-01').date()
end_date = pd.to_datetime('2023-12-31').date()
start_date_os = pd.to_datetime('2024-01-01').date()
end_date_os = pd.to_datetime('2025-10-01').date()
WARMUP_DAYS = 285
ticker_list = ['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD']#,'XRP-USD','AAVE-USD']
fast_mavg = 20
slow_mavg = 200
mavg_stepsize = 8
mavg_z_score_window = 126
entry_rolling_donchian_window = 56
exit_rolling_donchian_window = 28
use_donchian_exit_gate = False
long_only = True
use_coinbase_data = True
use_saved_files = True
saved_file_end_date = '2025-07-31'
volatility_window = 20
annual_trading_days = 365
rolling_cov_window = 20
annualized_target_volatility = 0.70
rolling_atr_window = 20
atr_multiplier = 2.0
use_specific_start_date = False
signal_start_date = None
initial_capital = 15000
cash_buffer_percentage = 0.10
transaction_cost_est = 0.001#0.0025
passive_trade_rate = 0.05
notional_threshold_pct = 0.05
cooldown_counter_threshold = 3
rolling_sharpe_window = 50
price_or_returns_calc = 'price'
moving_avg_type = 'exponential'
use_coinbase_data = True

ma_crossover_signal_weight = 0.9
donchian_signal_weight = 0.1
weighted_signal_ewm_window = 4
use_activation = False
tanh_activation_constant_dict = None
rolling_r2_window = 100
lower_r_sqr_limit = 0.45
upper_r_sqr_limit = 0.90
r2_smooth_window = 3
r2_confirm_days = 0
log_std_window = 14
coef_of_variation_window = 20
vol_of_vol_z_score_window = 126
vol_of_vol_p_min = 0.10
r2_strong_threshold = 0.75
use_specific_start_date = True
signal_start_date = pd.Timestamp('2022-04-01').date()

In [133]:
import itertools

def generate_target_volatility_stop_loss_atr_params():
    parameter_grid = {
        "target_volatility": np.arange(0.3, 1.1, 0.1).tolist(),
        "stop_loss_atr": [2]#np.arange(0.75, 3.1, 0.25).tolist()
    }
    keys, values = zip(*parameter_grid.items())
    for prod in itertools.product(*values):
        yield dict(zip(keys, prod))

In [212]:
def run_walk_forward_target_volatility_stop_loss_atr(start_date, end_date, ticker_list):

    start_date = pd.Timestamp(start_date).date()
    end_date = pd.Timestamp(end_date).date()
    perf_cols = ['sampling_category', 'start_date', 'end_date', 'target_volatility', 'stop_loss_atr', 'annualized_return', 'annualized_sharpe_ratio', 'calmar_ratio',
                 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate', 't_statistic', 'p_value', 'trade_count']
    ticker_perf_cols = ['annualized_return', 'annualized_sharpe_ratio', 'annualized_std_dev', 'max_drawdown']
    perf_cols.extend([f'{ticker}_{col}' for col in ticker_perf_cols for ticker in ticker_list])
    
    df_performance = pd.DataFrame(columns=perf_cols)
    
    IS_LEN = pd.DateOffset(months=18)
    OS_LEN = pd.DateOffset(months=6)
    start_date_is = start_date
    last_available_date = pd.Timestamp('2025-07-31').date()
    WARMUP_DAYS = 323
    while True:
        end_date_is = (start_date_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_date_os = (end_date_is + pd.Timedelta(days=1))
        end_date_os = (start_date_os + OS_LEN - pd.Timedelta(days=1)).date()
        fmt = "%Y-%m-%d"
        
        fields = [
            ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
            ("IS start",          start_date_is),
            ("IS end",            end_date_is),
            ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
            ("OS start",          start_date_os),
            ("OS end",            end_date_os),
        ]
        
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        ## Break Condition for While loop
        if end_date_os > end_date - pd.Timedelta(days=1):
            break

        if end_date_os > last_available_date:
            print('end_date_os > last_available_date')
            end_date_os = last_available_date
            fields = [
                ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
                ("IS start",          start_date_is),
                ("IS end",            end_date_is),
                ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
                ("OS start",          start_date_os),
                ("OS end",            end_date_os),
            ]
        
        print("Run Dates: ")
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
    
        for params in generate_target_volatility_stop_loss_atr_params():
            print(params)
            target_volatility = params['target_volatility']
            stop_loss_atr = params['stop_loss_atr']
            
            print(target_volatility, stop_loss_atr)

            ## In Sample Dataframe
            print('Pulling In Sample Data!!')
            df_is = tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol(
                start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, weighted_signal_ewm_window=weighted_signal_ewm_window,
                rolling_r2_window=rolling_r2_window, lower_r_sqr_limit=lower_r_sqr_limit, upper_r_sqr_limit=upper_r_sqr_limit, r2_smooth_window=r2_smooth_window, r2_confirm_days=r2_confirm_days,
                log_std_window=log_std_window, coef_of_variation_window=coef_of_variation_window, vol_of_vol_z_score_window=vol_of_vol_z_score_window, vol_of_vol_p_min=vol_of_vol_p_min, r2_strong_threshold=r2_strong_threshold,
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                moving_avg_type=moving_avg_type, long_only=long_only, price_or_returns_calc=price_or_returns_calc,
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=rolling_atr_window, atr_multiplier=stop_loss_atr,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate,
                notional_threshold_pct=notional_threshold_pct, cooldown_counter_threshold=cooldown_counter_threshold,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date, 
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage, annualized_target_volatility=target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=use_specific_start_date, signal_start_date=start_date_is)
            df_is = df_is[df_is.index >= pd.Timestamp(start_date_is).date()]
            
            print('Calculating In Sample Asset Returns!!')
            df_is = perf.calculate_asset_level_returns(df_is, end_date_is, ticker_list)

            ## In Sample Performance Metrics
            print('Getting In Sample Performance Metrics!!')
            row_parameters_is = {
                'sampling_category': 'in_sample',
                'start_date': start_date_is,
                'end_date': end_date_is,
                'target_volatility': target_volatility,
                'stop_loss_atr': stop_loss_atr
            }
            portfolio_perf_metrics_is = calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

            print('Getting In Sample Asset Performance!!')
            for ticker in ticker_list:
                ## In Sample
                ticker_perf_metrics_is = perf.calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_is = {key: ticker_perf_metrics_is[key] for key in ticker_perf_cols}
                ticker_perf_metrics_is = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_is.items()}
                portfolio_perf_metrics_is.update(ticker_perf_metrics_is)
            
            row_parameters_is.update(portfolio_perf_metrics_is)

            ## Assign in sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_is

        # Get Moving Average and Donchian Channel Weights with best performing in-sample Sharpe Ratio
        # in_sample_cond = (df_performance['sampling_category'] == 'in_sample')
        # date_cond = (df_performance['start_date'] == start_date_is)# & (df_performance['end_date'] == end_date_is)
        # walk_forward_run_cond = in_sample_cond & date_cond

        # ## Get Performing Parameters in the In-Sample period based on the Rolling R2 score
        # best_target_volatility = df_performance[walk_forward_run_cond].sort_values('annualized_sharpe_ratio', ascending=False)['target_volatility'].iloc[0]
        # best_stop_loss_atr = df_performance[walk_forward_run_cond].sort_values('annualized_sharpe_ratio', ascending=False)['stop_loss_atr'].iloc[0]
        
            ## Out of Sample Dataframe
            print('Pulling Out of Sample Data!!')
            df_os = tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol(
                start_date=start_date_os - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_os, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, weighted_signal_ewm_window=weighted_signal_ewm_window,
                rolling_r2_window=rolling_r2_window, lower_r_sqr_limit=lower_r_sqr_limit, upper_r_sqr_limit=upper_r_sqr_limit, r2_smooth_window=r2_smooth_window, r2_confirm_days=r2_confirm_days,
                log_std_window=log_std_window, coef_of_variation_window=coef_of_variation_window, vol_of_vol_z_score_window=vol_of_vol_z_score_window, vol_of_vol_p_min=vol_of_vol_p_min, r2_strong_threshold=r2_strong_threshold,
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                moving_avg_type=moving_avg_type, long_only=long_only, price_or_returns_calc=price_or_returns_calc,
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=rolling_atr_window, atr_multiplier=stop_loss_atr,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate,
                notional_threshold_pct=notional_threshold_pct, cooldown_counter_threshold=cooldown_counter_threshold,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date, 
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage, annualized_target_volatility=target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=use_specific_start_date, signal_start_date=start_date_os)
            df_os = df_os[df_os.index >= pd.Timestamp(start_date_os).date()]
                
            print('Calculating Out of Sample Asset Returns!!')
            df_os = perf.calculate_asset_level_returns(df_os, end_date_os, ticker_list)
            
            ## Out of Sample Performance Metrics
            print('Pulling Out of Sample Performance Metrics!!')
            row_parameters_os = {
                'sampling_category': 'out_sample',
                'start_date': start_date_os,
                'end_date': end_date_os,
                'target_volatility': target_volatility,
                'stop_loss_atr': stop_loss_atr
            }
            portfolio_perf_metrics_os = calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)
    
            print('Getting Out of Sample Asset Performance!!')
            for ticker in ticker_list:
                ## Out of Sample
                ticker_perf_metrics_os = perf.calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_os = {key: ticker_perf_metrics_os[key] for key in ticker_perf_cols}
                ticker_perf_metrics_os = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_os.items()}
                portfolio_perf_metrics_os.update(ticker_perf_metrics_os)
            
            row_parameters_os.update(portfolio_perf_metrics_os)
    
            ## Assign out of sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_os

        start_date_is = (start_date_is + OS_LEN).date()
        
    return df_performance

In [214]:
%%time
df_performance_target_volatility_stop_loss_atr_1 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2022-04-01', end_date='2024-04-01', ticker_list=ticker_list)

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
{'target_volatility': 0.3, 'stop_loss_atr': 2}
0.3 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio

In [215]:
df_performance_target_volatility_stop_loss_atr_1

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,0.3,2,-0.007785,-0.082063,-0.023833,0.323921,-0.326658,371 days,0.299270,0.124325,0.901104,1045.0,0.094353,0.005985,0.025355,-0.051515,0.051172,0.358984,-0.637817,-0.273166,-0.943822,0.065558,0.197513,0.086911,0.107054,0.141376,0.172773,-0.165003,-0.049923,-0.053151,-0.138734,-0.123869
1,out_sample,2023-10-01,2024-03-31,0.3,2,3.265245,4.564162,31.861014,0.325844,-0.102484,46 days,0.519126,3.340180,0.001016,491.0,0.219397,0.552043,0.162206,0.419421,0.600420,1.201597,2.884997,1.670193,2.952250,3.250213,0.149359,0.152947,0.068116,0.110611,0.151165,-0.034775,-0.072298,-0.020531,-0.023152,-0.028315
2,in_sample,2022-04-01,2023-09-30,0.4,2,-0.042103,-0.120859,-0.105261,0.397764,-0.399985,543 days,0.299270,0.035298,0.971855,1048.0,0.104272,0.011220,0.033482,-0.065273,0.033853,0.376830,-0.419962,-0.110533,-0.789520,-0.060096,0.237468,0.111013,0.139584,0.187740,0.184989,-0.194533,-0.067019,-0.069717,-0.181683,-0.158282
3,out_sample,2023-10-01,2024-03-31,0.4,2,5.062797,4.528530,37.507222,0.414667,-0.134982,46 days,0.519126,3.291619,0.001196,494.0,0.245218,0.739358,0.211575,0.515382,0.862709,1.151671,2.894772,1.797564,3.220527,3.346189,0.180418,0.198319,0.089608,0.123433,0.200878,-0.045149,-0.095449,-0.027456,-0.030829,-0.037460
4,in_sample,2022-04-01,2023-09-30,0.5,2,-0.074752,-0.152718,-0.164933,0.455088,-0.453230,543 days,0.301095,-0.027552,0.978029,1043.0,0.110630,0.008657,0.042335,-0.079483,0.016308,0.388962,-0.364991,-0.001947,-0.693383,-0.176926,0.261356,0.132018,0.169646,0.234305,0.194816,-0.213334,-0.085410,-0.085929,-0.223035,-0.185946
5,out_sample,2023-10-01,2024-03-31,0.5,2,6.412281,4.343482,38.271538,0.485075,-0.167547,46 days,0.519126,3.147942,0.001922,499.0,0.273771,0.747803,0.249714,0.583737,1.138996,1.166870,2.661903,1.814170,3.303557,3.375407,0.203438,0.219253,0.108534,0.134964,0.248662,-0.047227,-0.119019,-0.034213,-0.038193,-0.046614
6,in_sample,2022-04-01,2023-09-30,0.6,2,-0.117362,-0.207682,-0.231006,0.507960,-0.508049,543 days,0.302920,-0.111688,0.911112,1038.0,0.107388,0.004414,0.047546,-0.094977,-0.002273,0.364279,-0.345533,0.051122,-0.631067,-0.288512,0.276621,0.150442,0.197786,0.280663,0.206066,-0.225825,-0.103669,-0.102555,-0.265977,-0.213097
7,out_sample,2023-10-01,2024-03-31,0.6,2,7.314869,4.216353,39.446661,0.535582,-0.185437,48 days,0.519126,3.051391,0.002619,502.0,0.313237,0.735873,0.256001,0.608096,1.349062,1.242157,2.533092,1.683157,3.316270,3.328442,0.221690,0.228189,0.121068,0.139525,0.286950,-0.047227,-0.128738,-0.040544,-0.042289,-0.055150
8,in_sample,2022-04-01,2023-09-30,0.7,2,-0.159040,-0.269948,-0.287938,0.548006,-0.552342,543 days,0.302920,-0.197896,0.843200,1032.0,0.113213,0.002753,0.050038,-0.123073,-0.017272,0.386499,-0.313640,0.076139,-0.702977,-0.363615,0.282347,0.166757,0.223797,0.306406,0.218334,-0.225825,-0.112984,-0.117176,-0.301264,-0.237178
9,out_sample,2023-10-01,2024-03-31,0.7,2,7.614356,4.118735,39.204763,0.560591,-0.194220,48 days,0.519126,2.979364,0.003283,503.0,0.333317,0.724234,0.246967,0.603370,1.463034,1.251189,2.461010,1.533928,3.266199,3.296488,0.236120,0.232226,0.128229,0.140764,0.307736,-0.047629,-0.132322,-0.043780,-0.046078,-0.059450


In [216]:
df_performance_target_volatility_stop_loss_atr_1.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Stop_Loss_ATR_Performance-2022-04-01-2024-04-01.pickle')

In [217]:
%%time
df_performance_target_volatility_stop_loss_atr_2 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2022-10-01', end_date='2024-10-01',ticker_list=ticker_list)

Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
Run Dates: 
Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
{'target_volatility': 0.3, 'stop_loss_atr': 2}
0.3 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio

In [218]:
df_performance_target_volatility_stop_loss_atr_2.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Stop_Loss_ATR_Performance-2022-10-01-2024-10-01.pickle')

In [219]:
%%time
df_performance_target_volatility_stop_loss_atr_3 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2023-04-01', end_date='2025-04-01',ticker_list=ticker_list)

Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
Run Dates: 
Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
{'target_volatility': 0.3, 'stop_loss_atr': 2}
0.3 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio

In [220]:
df_performance_target_volatility_stop_loss_atr_3.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Stop_Loss_ATR_Performance-2023-04-01-2025-04-01.pickle')

In [221]:
%%time
df_performance_target_volatility_stop_loss_atr_4 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2023-10-01', end_date='2025-10-01',ticker_list=ticker_list)

Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-09-30
end_date_os > last_available_date
Run Dates: 
Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-07-31
{'target_volatility': 0.3, 'stop_loss_atr': 2}
0.3 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash 

In [222]:
df_performance_target_volatility_stop_loss_atr_4.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Stop_Loss_ATR_Performance-2023-10-01-2025-10-01.pickle')

In [232]:
df_performance_target_volatility_stop_loss_atr = pd.concat([df_performance_target_volatility_stop_loss_atr_1, df_performance_target_volatility_stop_loss_atr_2,
                                                            df_performance_target_volatility_stop_loss_atr_3, df_performance_target_volatility_stop_loss_atr_4], axis=0, ignore_index=True)

In [234]:
df_performance_target_volatility_stop_loss_atr.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Stop_Loss_ATR_Performance-2022-04-01-2025-10-01.pickle')

In [236]:
df_performance_target_volatility_stop_loss_atr

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,0.3,2,-0.007785,-0.082063,-0.023833,0.323921,-0.326658,371 days,0.299270,0.124325,0.901104,1045.0,0.094353,0.005985,0.025355,-0.051515,0.051172,0.358984,-0.637817,-0.273166,-0.943822,0.065558,0.197513,0.086911,0.107054,1.413763e-01,0.172773,-0.165003,-0.049923,-0.053151,-0.138734,-0.123869
1,out_sample,2023-10-01,2024-03-31,0.3,2,3.265245,4.564162,31.861014,0.325844,-0.102484,46 days,0.519126,3.340180,0.001016,491.0,0.219397,0.552043,0.162206,0.419421,0.600420,1.201597,2.884997,1.670193,2.952250,3.250213,0.149359,0.152947,0.068116,1.106114e-01,0.151165,-0.034775,-0.072298,-0.020531,-0.023152,-0.028315
2,in_sample,2022-04-01,2023-09-30,0.4,2,-0.042103,-0.120859,-0.105261,0.397764,-0.399985,543 days,0.299270,0.035298,0.971855,1048.0,0.104272,0.011220,0.033482,-0.065273,0.033853,0.376830,-0.419962,-0.110533,-0.789520,-0.060096,0.237468,0.111013,0.139584,1.877395e-01,0.184989,-0.194533,-0.067019,-0.069717,-0.181683,-0.158282
3,out_sample,2023-10-01,2024-03-31,0.4,2,5.062797,4.528530,37.507222,0.414667,-0.134982,46 days,0.519126,3.291619,0.001196,494.0,0.245218,0.739358,0.211575,0.515382,0.862709,1.151671,2.894772,1.797564,3.220527,3.346189,0.180418,0.198319,0.089608,1.234330e-01,0.200878,-0.045149,-0.095449,-0.027456,-0.030829,-0.037460
4,in_sample,2022-04-01,2023-09-30,0.5,2,-0.074752,-0.152718,-0.164933,0.455088,-0.453230,543 days,0.301095,-0.027552,0.978029,1043.0,0.110630,0.008657,0.042335,-0.079483,0.016308,0.388962,-0.364991,-0.001947,-0.693383,-0.176926,0.261356,0.132018,0.169646,2.343046e-01,0.194816,-0.213334,-0.085410,-0.085929,-0.223035,-0.185946
5,out_sample,2023-10-01,2024-03-31,0.5,2,6.412281,4.343482,38.271538,0.485075,-0.167547,46 days,0.519126,3.147942,0.001922,499.0,0.273771,0.747803,0.249714,0.583737,1.138996,1.166870,2.661903,1.814170,3.303557,3.375407,0.203438,0.219253,0.108534,1.349638e-01,0.248662,-0.047227,-0.119019,-0.034213,-0.038193,-0.046614
6,in_sample,2022-04-01,2023-09-30,0.6,2,-0.117362,-0.207682,-0.231006,0.507960,-0.508049,543 days,0.302920,-0.111688,0.911112,1038.0,0.107388,0.004414,0.047546,-0.094977,-0.002273,0.364279,-0.345533,0.051122,-0.631067,-0.288512,0.276621,0.150442,0.197786,2.806627e-01,0.206066,-0.225825,-0.103669,-0.102555,-0.265977,-0.213097
7,out_sample,2023-10-01,2024-03-31,0.6,2,7.314869,4.216353,39.446661,0.535582,-0.185437,48 days,0.519126,3.051391,0.002619,502.0,0.313237,0.735873,0.256001,0.608096,1.349062,1.242157,2.533092,1.683157,3.316270,3.328442,0.221690,0.228189,0.121068,1.395254e-01,0.286950,-0.047227,-0.128738,-0.040544,-0.042289,-0.055150
8,in_sample,2022-04-01,2023-09-30,0.7,2,-0.159040,-0.269948,-0.287938,0.548006,-0.552342,543 days,0.302920,-0.197896,0.843200,1032.0,0.113213,0.002753,0.050038,-0.123073,-0.017272,0.386499,-0.313640,0.076139,-0.702977,-0.363615,0.282347,0.166757,0.223797,3.064064e-01,0.218334,-0.225825,-0.112984,-0.117176,-0.301264,-0.237178
9,out_sample,2023-10-01,2024-03-31,0.7,2,7.614356,4.118735,39.204763,0.560591,-0.194220,48 days,0.519126,2.979364,0.003283,503.0,0.333317,0.724234,0.246967,0.603370,1.463034,1.251189,2.461010,1.533928,3.266199,3.296488,0.236120,0.232226,0.128229,1.407643e-01,0.307736,-0.047629,-0.132322,-0.043780,-0.046078,-0.059450


In [238]:
df_performance_target_volatility_stop_loss_atr['vol_tracking_error'] = (np.abs(df_performance_target_volatility_stop_loss_atr['annualized_std_dev'] - df_performance_target_volatility_stop_loss_atr['target_volatility'])/
                                                                       df_performance_target_volatility_stop_loss_atr['target_volatility'])

In [240]:
in_sample_cond = (df_performance_target_volatility_stop_loss_atr.sampling_category == 'in_sample')
df_performance_target_volatility_stop_loss_atr_is = df_performance_target_volatility_stop_loss_atr[in_sample_cond].reset_index(drop=True)
df_performance_target_volatility_stop_loss_atr_os = df_performance_target_volatility_stop_loss_atr[~in_sample_cond].reset_index(drop=True)

In [254]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'annualized_std_dev':['median','mean','std'],
            'vol_tracking_error':['median','mean','std'],
            'trade_count':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}

In [256]:
df_performance_target_volatility_stop_loss_atr_is.groupby(['target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)

annualized_sharpe_ratio                      \
                                   median      mean       std   
target_volatility                                               
0.3                              1.798617  1.415624  1.013034   
0.4                              1.818428  1.408951  1.033023   
0.5                              1.752078  1.349157  1.013800   
0.6                              1.730304  1.301222  1.013957   
0.7                              1.703517  1.247243  1.015121   
0.8                              1.685575  1.218691  1.008975   
0.9                              1.657693  1.183648  1.006444   
1.0                              1.632965  1.162098  0.995373   

                  annualized_return                     max_drawdown  \
                             median      mean       std       median   
target_volatility                                                      
0.3                        0.539738  0.492151  0.373408    -0.172671   
0.4                        0.707046  0.626071  0.488626    -0.224760   
0.5                        0.806364  0.701299  0.561194    -0.267500   
0.6                        0.884099  0.742813  0.609224    -0.295831   
0.7                        0.909165  0.740905  0.628350    -0.311830   
0.8                        0.913986  0.734255  0.636253    -0.317014   
0.9                        0.899195  0.711217  0.630375    -0.320708   
1.0                        0.884976  0.696198  0.622840    -0.321327   

                                      annualized_std_dev                      \
                       mean       std             median      mean       std   
target_volatility                                                              
0.3               -0.203526  0.084014           0.289029  0.285884  0.045105   
0.4               -0.255329  0.100634           0.363065  0.358073  0.049840   
0.5               -0.300038  0.105901           0.423187  0.415100  0.051111   
0.6               -0.331443  0.123430           0.470527  0.459595  0.057944   
0.7               -0.349894  0.144244           0.496350  0.487050  0.067813   
0.8               -0.356285  0.159646           0.508556  0.501186  0.075998   
0.9               -0.362949  0.161851           0.511725  0.505650  0.079608   
1.0               -0.365071  0.159960           0.513445  0.508045  0.081225   

                  vol_tracking_error                     trade_count           \
                              median      mean       std      median     mean   
target_volatility                                                               
0.3                         0.118664  0.129148  0.057606      1065.0  1102.25   
0.4                         0.093044  0.108320  0.120540      1069.5  1099.75   
0.5                         0.153626  0.169801  0.102222      1067.0  1097.00   
0.6                         0.215789  0.234009  0.096574      1069.5  1103.00   
0.7                         0.290928  0.304215  0.096875      1063.0  1101.00   
0.8                         0.364306  0.373517  0.094997      1073.0  1111.75   
0.9                         0.431417  0.438167  0.088453      1076.0  1115.50   
1.0                         0.486555  0.491955  0.081225      1074.5  1113.25   

                              BTC-USD_annualized_sharpe_ratio            \
                          std                          median      mean   
target_volatility                                                         
0.3                129.685196                        0.384646  0.576742   
0.4                122.355697                        0.402808  0.585591   
0.5                119.498954                        0.439909  0.608247   
0.6                126.039676                        0.470145  0.650179   
0.7                129.524772                        0.486189  0.668028   
0.8                129.288760                        0.511750  0.696283   
0.9                128.958650                        0.521553

In [258]:
df_performance_target_volatility_stop_loss_atr_os.groupby(['target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)

annualized_sharpe_ratio                      \
                                   median      mean       std   
target_volatility                                               
0.6                              1.242429  1.463302  2.198085   
0.7                              1.230914  1.437955  2.130764   
0.8                              1.238481  1.436271  2.085125   
0.5                              1.185975  1.427316  2.339894   
0.9                              1.218049  1.419874  2.064700   
0.4                              1.099577  1.413108  2.472449   
1.0                              1.210733  1.408781  2.050975   
0.3                              0.921416  1.298876  2.520086   

                  annualized_return                     max_drawdown  \
                             median      mean       std       median   
target_volatility                                                      
0.6                        0.429331  2.008158  3.556371    -0.164375   
0.7                        0.435442  2.084660  3.703339    -0.167232   
0.8                        0.447712  2.097737  3.711777    -0.166999   
0.5                        0.391968  1.764225  3.120185    -0.149078   
0.9                        0.446384  2.075120  3.667635    -0.166999   
0.4                        0.353577  1.414818  2.456948    -0.120155   
1.0                        0.451147  2.050181  3.612057    -0.166999   
0.3                        0.248702  0.919324  1.582894    -0.091812   

                                      annualized_std_dev                      \
                       mean       std             median      mean       std   
target_volatility                                                              
0.6               -0.153012  0.040926           0.503752  0.463684  0.108844   
0.7               -0.156636  0.043754           0.526472  0.484081  0.118008   
0.8               -0.156820  0.043783           0.533603  0.492582  0.121018   
0.5               -0.143375  0.036431           0.440558  0.415346  0.085097   
0.9               -0.156799  0.043820           0.534836  0.495103  0.119164   
0.4               -0.123257  0.026469           0.356578  0.352430  0.055654   
1.0               -0.156703  0.043992           0.535401  0.497694  0.115048   
0.3               -0.095017  0.017217           0.269172  0.274517  0.038933   

                  vol_tracking_error                     trade_count          \
                              median      mean       std      median    mean   
target_volatility                                                              
0.6                         0.160414  0.227193  0.181407       313.0  294.00   
0.7                         0.247897  0.308456  0.168583       308.5  291.50   
0.8                         0.332996  0.384272  0.151272       304.5  292.00   
0.5                         0.118885  0.169308  0.170193       308.5  291.25   
0.9                         0.405737  0.449886  0.132404       304.0  293.25   
0.4                         0.108555  0.137257  0.114440       315.0  293.25   
1.0                         0.464599  0.502306  0.115048       305.5  294.00   
0.3                         0.111297  0.128017  0.067907       318.0  294.25   

                              BTC-USD_annualized_sharpe_ratio            \
                          std                          median      mean   
target_volatility                                                         
0.6                198.284308                        1.130718  0.628241   
0.7                198.246480                        1.134827  0.640485   
0.8                200.640641                        1.146097  0.669207   
0.5                195.322596                        1.044262  0.555692   
0.9                200.629634                        1.146291  0.673581   
0.4                194.958756                        0.952385  0.477428   
1.0                200.314752                        1.146322  0.673602 

In [252]:
df_performance_target_volatility_stop_loss_atr_os.groupby(['start_date','target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)

annualized_sharpe_ratio                \
                                              median      mean std   
start_date target_volatility                                         
2023-10-01 0.3                              4.564162  4.564162 NaN   
           0.4                              4.528530  4.528530 NaN   
           0.5                              4.343482  4.343482 NaN   
           0.6                              4.216353  4.216353 NaN   
           0.7                              4.118735  4.118735 NaN   
           0.8                              4.067880  4.067880 NaN   
           0.9                              4.035410  4.035410 NaN   
           1.0                              4.005673  4.005673 NaN   
2025-04-01 0.4                              2.158965  2.158965 NaN   
           0.5                              2.156270  2.156270 NaN   
           0.6                              2.101423  2.101423 NaN   
           0.7                              2.008809  2.008809 NaN   
           0.8                              1.959526  1.959526 NaN   
           0.9                              1.915911  1.915911 NaN   
           1.0                              1.901278  1.901278 NaN   
           0.3                              1.871024  1.871024 NaN   
2024-10-01 1.0                              0.520188  0.520188 NaN   
           0.9                              0.520188  0.520188 NaN   
           0.8                              0.517436  0.517436 NaN   
           0.7                              0.453019  0.453019 NaN   
           0.6                              0.383434  0.383434 NaN   
           0.5                              0.215679  0.215679 NaN   
           0.4                              0.040189  0.040189 NaN   
           0.3                             -0.028192 -0.028192 NaN   
2024-04-01 0.9                             -0.792014 -0.792014 NaN   
           1.0                             -0.792014 -0.792014 NaN   
           0.8                             -0.799757 -0.799757 NaN   
           0.7                             -0.828742 -0.828742 NaN   
           0.6                             -0.848001 -0.848001 NaN   
           0.5                             -1.006168 -1.006168 NaN   
           0.4                             -1.075253 -1.075253 NaN   
           0.3                             -1.211492 -1.211492 NaN   

                             annualized_return               max_drawdown  \
                                        median      mean std       median   
start_date target_volatility                                                
2023-10-01 0.3                        3.265245  3.265245 NaN    -0.102484   
           0.4                        5.062797  5.062797 NaN    -0.134982   
           0.5                        6.412281  6.412281 NaN    -0.167547   
           0.6                        7.314869  7.314869 NaN    -0.185437   
           0.7                        7.614356  7.614356 NaN    -0.194220   
           0.8                        7.640946  7.640946 NaN    -0.194951   
           0.9                        7.552132  7.552132 NaN    -0.194951   
           1.0                        7.442854  7.442854 NaN    -0.194951   
2025-04-01 0.4                        0.687070  0.687070 NaN    -0.097866   
           0.5                        0.718000  0.718000 NaN    -0.097864   
           0.6                        0.727909  0.727909 NaN    -0.097863   
           0.7                        0.707708  0.707708 NaN    -0.097861   
           0.8                        0.700466  0.700466 NaN    -0.098328   
           0.9                        0.696428  0.696428 NaN    -0.098245   
           1.0                        0.705954  0.705954 NaN    -0.097860   
           0.3                        0.477256  0.477256 NaN    -0.080609   
2024-10-01 1.0                        0.196339  0.196339 NaN    -0.185281   
           0.9                        

In [250]:
df_performance_target_volatility_stop_loss_atr_os[df_performance_target_volatility_stop_loss_atr_os.start_date == pd.Timestamp('2025-04-01').date()]

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown,vol_tracking_error
24,out_sample,2025-04-01,2025-07-31,0.3,2,0.477256,1.871024,5.920640,0.233879,-0.080609,39 days,0.401639,1.228405,0.221679,247.0,0.177058,0.334362,0.045059,0.0,-0.000530,0.916524,2.889545,-0.068061,-inf,-44.152807,0.165082,0.103907,0.061985,5.798764e-27,0.001374,-0.055532,-0.021789,-0.012071,0.0,-0.000377,0.220403
25,out_sample,2025-04-01,2025-07-31,0.4,2,0.687070,2.158965,7.020500,0.281899,-0.097866,39 days,0.409836,1.369743,0.173304,234.0,0.250691,0.417817,0.063363,0.0,-0.000707,1.186656,2.998402,0.220791,-inf,-33.232498,0.193735,0.125792,0.082242,6.344900e-27,0.001832,-0.067799,-0.027245,-0.014936,0.0,-0.000503,0.295251
26,out_sample,2025-04-01,2025-07-31,0.5,2,0.718000,2.156270,7.336686,0.295194,-0.097864,39 days,0.409836,1.363266,0.175330,231.0,0.259156,0.410480,0.086877,0.0,-0.000884,1.224417,2.752985,0.453697,-inf,-26.680277,0.194630,0.135222,0.102109,6.344903e-27,0.002289,-0.067797,-0.029624,-0.017658,0.0,-0.000629,0.409612
27,out_sample,2025-04-01,2025-07-31,0.6,2,0.727909,2.101423,7.438077,0.307227,-0.097863,39 days,0.409836,1.326820,0.187067,231.0,0.259693,0.401150,0.110643,0.0,-0.001061,1.226099,2.509516,0.610340,-inf,-22.312108,0.194813,0.145722,0.122083,6.344903e-27,0.002747,-0.067796,-0.034430,-0.021226,0.0,-0.000755,0.487955
28,out_sample,2025-04-01,2025-07-31,0.7,2,0.707708,2.008809,7.231795,0.314683,-0.097861,39 days,0.409836,1.270421,0.206372,230.0,0.260174,0.391858,0.111644,0.0,-0.001652,1.227491,2.283074,0.602556,-inf,-21.165991,0.194997,0.157349,0.126174,6.344903e-27,0.002930,-0.067794,-0.040157,-0.023018,0.0,-0.000858,0.550453
29,out_sample,2025-04-01,2025-07-31,0.8,2,0.700466,1.959526,7.123774,0.320015,-0.098328,56 days,0.418033,1.239859,0.217427,230.0,0.260263,0.391183,0.108116,0.0,-0.001897,1.227778,2.134316,0.570679,-inf,-20.793357,0.195025,0.168850,0.126477,6.344903e-27,0.002997,-0.067793,-0.045880,-0.023018,0.0,-0.000858,0.599981
30,out_sample,2025-04-01,2025-07-31,0.9,2,0.696428,1.915911,7.088699,0.325884,-0.098245,56 days,0.418033,1.212446,0.227704,229.0,0.260271,0.392073,0.104629,0.0,-0.002135,1.227827,2.006220,0.539085,-inf,-20.389098,0.195024,0.180970,0.126815,6.344903e-27,0.003070,-0.067793,-0.051601,-0.023018,0.0,-0.000858,0.637907
31,out_sample,2025-04-01,2025-07-31,1.0,2,0.705954,1.901278,7.213894,0.335118,-0.097860,39 days,0.418033,1.201754,0.231806,232.0,0.260284,0.408407,0.101720,0.0,-0.002360,1.227889,1.974417,0.512656,-inf,-19.964041,0.195024,0.192089,0.127137,6.344903e-27,0.003149,-0.067793,-0.056848,-0.023018,0.0,-0.000895,0.664882


In [109]:
df_performance_target_volatility_stop_loss_atr.sort_values('annualized_sharpe_ratio', ascending=False)

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
80,out_sample,2023-10-01,2024-03-31,0.3,1.00,2.842807,4.379796,32.100724,0.316697,-0.088559,46 days,0.513661,3.213621,0.001551,465.0,0.157372,0.536167,0.153735,0.400178,0.594670,0.837959,2.818479,1.554447,2.859836,3.223198,0.154304,0.162295,0.069394,1.143538e-01,0.154400,-0.054665,-0.072361,-0.020566,-0.020704,-0.028323
82,in_sample,2022-10-01,2024-03-31,0.3,1.00,0.882540,2.244391,7.828796,0.308037,-0.112730,191 days,0.394161,2.965831,0.003150,1213.0,0.243025,0.156448,0.082307,0.129156,0.276900,1.449974,1.080562,0.400793,0.900147,1.462774,0.158241,0.123519,0.104925,1.098085e-01,0.190427,-0.056687,-0.072140,-0.053151,-0.072996,-0.068904
85,in_sample,2022-10-01,2024-03-31,0.3,1.75,0.928122,2.224836,5.486285,0.323551,-0.169171,191 days,0.405109,2.930667,0.003524,1277.0,0.273140,0.174080,0.086478,0.130026,0.245616,1.558230,1.227374,0.444148,0.886893,1.272983,0.158130,0.118545,0.102583,1.080390e-01,0.179376,-0.059751,-0.072077,-0.053151,-0.055147,-0.106405
92,in_sample,2022-10-01,2024-03-31,0.4,1.00,1.106049,2.208530,7.460950,0.379126,-0.148245,191 days,0.392336,2.881288,0.004116,1204.0,0.267977,0.206274,0.105978,0.154373,0.336714,1.379134,1.218467,0.525390,0.977981,1.567682,0.188086,0.158331,0.137050,1.325257e-01,0.220081,-0.088744,-0.095241,-0.069717,-0.090862,-0.089718
95,in_sample,2022-10-01,2024-03-31,0.4,1.75,1.178940,2.203068,5.337974,0.399956,-0.220859,191 days,0.405109,2.865143,0.004328,1261.0,0.310821,0.229760,0.113121,0.156064,0.293530,1.523329,1.361264,0.581106,0.962614,1.350472,0.188031,0.152154,0.133926,1.312382e-01,0.207534,-0.068161,-0.095160,-0.069717,-0.073041,-0.136662
87,in_sample,2022-10-01,2024-03-31,0.3,2.25,0.901533,2.148654,4.767346,0.302114,-0.189106,191 days,0.406934,2.834197,0.004764,1412.0,0.279555,0.174116,0.077627,0.136055,0.223489,1.596247,1.227715,0.347385,0.938498,1.131465,0.157578,0.117545,0.102740,9.363821e-02,0.179319,-0.059843,-0.072077,-0.053151,-0.050990,-0.130136
86,in_sample,2022-10-01,2024-03-31,0.3,2.00,0.896914,2.147323,4.831232,0.325336,-0.185649,191 days,0.406934,2.833323,0.004777,1288.0,0.273098,0.174080,0.077633,0.136521,0.223539,1.557934,1.227374,0.347455,0.943144,1.131726,0.158135,0.117547,0.102741,1.087842e-01,0.180087,-0.059796,-0.072077,-0.053151,-0.050990,-0.130136
88,in_sample,2022-10-01,2024-03-31,0.3,2.50,0.904861,2.142333,4.907095,0.304133,-0.184399,191 days,0.406934,2.825131,0.004899,1419.0,0.279560,0.168462,0.077627,0.135887,0.227726,1.596307,1.172367,0.347385,0.936757,1.146285,0.157575,0.117542,0.102740,9.364448e-02,0.179698,-0.059837,-0.072077,-0.053151,-0.050990,-0.125610
89,in_sample,2022-10-01,2024-03-31,0.3,2.75,0.903762,2.139768,4.882428,0.304240,-0.185105,191 days,0.406934,2.821919,0.004947,1419.0,0.279560,0.168462,0.077627,0.135887,0.227726,1.596307,1.172367,0.347385,0.936758,1.146285,0.157575,0.117542,0.102740,9.364449e-02,0.179698,-0.059837,-0.072077,-0.053151,-0.050990,-0.125610
102,in_sample,2022-10-01,2024-03-31,0.5,1.00,1.239860,2.138850,7.273950,0.432156,-0.170452,190 days,0.394161,2.774302,0.005721,1197.0,0.284806,0.207683,0.126324,0.170362,0.394445,1.363882,1.112344,0.590139,0.976295,1.654970,0.204629,0.177021,0.166231,1.534032e-01,0.245680,-0.106515,-0.118793,-0.085929,-0.105949,-0.100138


## Walk Forward Analysis for Target Volatility Micro Grid

In [265]:
np.arange(0.4, 0.75, 0.05).tolist()

[0.4, 0.45, 0.5, 0.55, 0.6, 0.6499999999999999, 0.7]

In [267]:
import itertools

def generate_target_volatility_stop_loss_atr_params():
    parameter_grid = {
        "target_volatility": np.arange(0.4, 0.75, 0.05).tolist(),
        "stop_loss_atr": [2]#np.arange(0.75, 3.1, 0.25).tolist()
    }
    keys, values = zip(*parameter_grid.items())
    for prod in itertools.product(*values):
        yield dict(zip(keys, prod))

In [331]:
def run_walk_forward_target_volatility_stop_loss_atr(start_date, end_date, ticker_list):

    start_date = pd.Timestamp(start_date).date()
    end_date = pd.Timestamp(end_date).date()
    perf_cols = ['sampling_category', 'start_date', 'end_date', 'target_volatility', 'stop_loss_atr', 'annualized_return', 'annualized_sharpe_ratio', 'calmar_ratio',
                 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate', 't_statistic', 'p_value', 'trade_count']
    ticker_perf_cols = ['annualized_return', 'annualized_sharpe_ratio', 'annualized_std_dev', 'max_drawdown']
    perf_cols.extend([f'{ticker}_{col}' for col in ticker_perf_cols for ticker in ticker_list])
    
    df_performance = pd.DataFrame(columns=perf_cols)
    
    IS_LEN = pd.DateOffset(months=18)
    OS_LEN = pd.DateOffset(months=6)
    start_date_is = start_date
    last_available_date = pd.Timestamp('2025-07-31').date()
    WARMUP_DAYS = 323
    while True:
        end_date_is = (start_date_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_date_os = (end_date_is + pd.Timedelta(days=1))
        end_date_os = (start_date_os + OS_LEN - pd.Timedelta(days=1)).date()
        fmt = "%Y-%m-%d"
        
        fields = [
            ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
            ("IS start",          start_date_is),
            ("IS end",            end_date_is),
            ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
            ("OS start",          start_date_os),
            ("OS end",            end_date_os),
        ]
        
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        ## Break Condition for While loop
        if end_date_os > end_date - pd.Timedelta(days=1):
            break

        if end_date_os > last_available_date:
            print('end_date_os > last_available_date')
            end_date_os = last_available_date
            fields = [
                ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
                ("IS start",          start_date_is),
                ("IS end",            end_date_is),
                ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
                ("OS start",          start_date_os),
                ("OS end",            end_date_os),
            ]
        
        print("Run Dates: ")
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
    
        for params in generate_target_volatility_stop_loss_atr_params():
            print(params)
            target_volatility = params['target_volatility']
            stop_loss_atr = params['stop_loss_atr']
            
            print(target_volatility, stop_loss_atr)

            ## In Sample Dataframe
            print('Pulling In Sample Data!!')
            df_is = tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol(
                start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, weighted_signal_ewm_window=weighted_signal_ewm_window,
                rolling_r2_window=rolling_r2_window, lower_r_sqr_limit=lower_r_sqr_limit, upper_r_sqr_limit=upper_r_sqr_limit, r2_smooth_window=r2_smooth_window, r2_confirm_days=r2_confirm_days,
                log_std_window=log_std_window, coef_of_variation_window=coef_of_variation_window, vol_of_vol_z_score_window=vol_of_vol_z_score_window, vol_of_vol_p_min=vol_of_vol_p_min, r2_strong_threshold=r2_strong_threshold,
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                moving_avg_type=moving_avg_type, long_only=long_only, price_or_returns_calc=price_or_returns_calc,
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=rolling_atr_window, atr_multiplier=stop_loss_atr,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate,
                notional_threshold_pct=notional_threshold_pct, cooldown_counter_threshold=cooldown_counter_threshold,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date, 
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage, annualized_target_volatility=target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=use_specific_start_date, signal_start_date=start_date_is)
            df_is = df_is[df_is.index >= pd.Timestamp(start_date_is).date()]
            
            print('Calculating In Sample Asset Returns!!')
            df_is = perf.calculate_asset_level_returns(df_is, end_date_is, ticker_list)

            ## In Sample Performance Metrics
            print('Getting In Sample Performance Metrics!!')
            row_parameters_is = {
                'sampling_category': 'in_sample',
                'start_date': start_date_is,
                'end_date': end_date_is,
                'target_volatility': target_volatility,
                'stop_loss_atr': stop_loss_atr
            }
            portfolio_perf_metrics_is = calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

            print('Getting In Sample Asset Performance!!')
            for ticker in ticker_list:
                ## In Sample
                ticker_perf_metrics_is = perf.calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_is = {key: ticker_perf_metrics_is[key] for key in ticker_perf_cols}
                ticker_perf_metrics_is = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_is.items()}
                portfolio_perf_metrics_is.update(ticker_perf_metrics_is)
            
            row_parameters_is.update(portfolio_perf_metrics_is)

            ## Assign in sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_is

        # Get Moving Average and Donchian Channel Weights with best performing in-sample Sharpe Ratio
        # in_sample_cond = (df_performance['sampling_category'] == 'in_sample')
        # date_cond = (df_performance['start_date'] == start_date_is)# & (df_performance['end_date'] == end_date_is)
        # walk_forward_run_cond = in_sample_cond & date_cond

        # ## Get Performing Parameters in the In-Sample period based on the Rolling R2 score
        # best_target_volatility = df_performance[walk_forward_run_cond].sort_values('annualized_sharpe_ratio', ascending=False)['target_volatility'].iloc[0]
        # best_stop_loss_atr = df_performance[walk_forward_run_cond].sort_values('annualized_sharpe_ratio', ascending=False)['stop_loss_atr'].iloc[0]
        
            ## Out of Sample Dataframe
            print('Pulling Out of Sample Data!!')
            df_os = tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol(
                start_date=start_date_os - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_os, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, weighted_signal_ewm_window=weighted_signal_ewm_window,
                rolling_r2_window=rolling_r2_window, lower_r_sqr_limit=lower_r_sqr_limit, upper_r_sqr_limit=upper_r_sqr_limit, r2_smooth_window=r2_smooth_window, r2_confirm_days=r2_confirm_days,
                log_std_window=log_std_window, coef_of_variation_window=coef_of_variation_window, vol_of_vol_z_score_window=vol_of_vol_z_score_window, vol_of_vol_p_min=vol_of_vol_p_min, r2_strong_threshold=r2_strong_threshold,
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                moving_avg_type=moving_avg_type, long_only=long_only, price_or_returns_calc=price_or_returns_calc,
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=rolling_atr_window, atr_multiplier=stop_loss_atr,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate,
                notional_threshold_pct=notional_threshold_pct, cooldown_counter_threshold=cooldown_counter_threshold,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date, 
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage, annualized_target_volatility=target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=use_specific_start_date, signal_start_date=start_date_os)
            df_os = df_os[df_os.index >= pd.Timestamp(start_date_os).date()]
                
            print('Calculating Out of Sample Asset Returns!!')
            df_os = perf.calculate_asset_level_returns(df_os, end_date_os, ticker_list)
            
            ## Out of Sample Performance Metrics
            print('Pulling Out of Sample Performance Metrics!!')
            row_parameters_os = {
                'sampling_category': 'out_sample',
                'start_date': start_date_os,
                'end_date': end_date_os,
                'target_volatility': target_volatility,
                'stop_loss_atr': stop_loss_atr
            }
            portfolio_perf_metrics_os = calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)
    
            print('Getting Out of Sample Asset Performance!!')
            for ticker in ticker_list:
                ## Out of Sample
                ticker_perf_metrics_os = perf.calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_os = {key: ticker_perf_metrics_os[key] for key in ticker_perf_cols}
                ticker_perf_metrics_os = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_os.items()}
                portfolio_perf_metrics_os.update(ticker_perf_metrics_os)
            
            row_parameters_os.update(portfolio_perf_metrics_os)
    
            ## Assign out of sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_os

        start_date_is = (start_date_is + OS_LEN).date()
        
    return df_performance

In [333]:
%%time
df_performance_target_volatility_micro_1 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2022-04-01', end_date='2024-04-01', ticker_list=ticker_list)

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
{'target_volatility': 0.4, 'stop_loss_atr': 2}
0.4 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio

In [334]:
df_performance_target_volatility_micro_1

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,0.40,2,-0.042103,-0.120859,-0.105261,0.397764,-0.399985,543 days,0.299270,0.035298,0.971855,1048.0,0.104272,0.011220,0.033482,-0.065273,0.033853,0.376830,-0.419962,-0.110533,-0.789520,-0.060096,0.237468,0.111013,0.139584,0.187740,0.184989,-0.194533,-0.067019,-0.069717,-0.181683,-0.158282
1,out_sample,2023-10-01,2024-03-31,0.40,2,5.062797,4.528530,37.507222,0.414667,-0.134982,46 days,0.519126,3.291619,0.001196,494.0,0.245218,0.739358,0.211575,0.515382,0.862709,1.151671,2.894772,1.797564,3.220527,3.346189,0.180418,0.198319,0.089608,0.123433,0.200878,-0.045149,-0.095449,-0.027456,-0.030829,-0.037460
2,in_sample,2022-04-01,2023-09-30,0.45,2,-0.050502,-0.111321,-0.119184,0.429003,-0.423735,543 days,0.297445,0.033085,0.973619,1039.0,0.116199,0.011541,0.037903,-0.071947,0.025523,0.422577,-0.368814,-0.051284,-0.733602,-0.116706,0.252917,0.122545,0.154722,0.211071,0.189765,-0.204613,-0.076348,-0.078182,-0.202557,-0.172095
3,out_sample,2023-10-01,2024-03-31,0.45,2,5.824276,4.459586,38.589493,0.450482,-0.150929,46 days,0.519126,3.235811,0.001441,493.0,0.258946,0.763862,0.231678,0.552705,1.002997,1.158821,2.810420,1.815455,3.276705,3.369353,0.191620,0.210601,0.099153,0.129433,0.225442,-0.046210,-0.106850,-0.030961,-0.034753,-0.041879
4,in_sample,2022-04-01,2023-09-30,0.50,2,-0.074752,-0.152718,-0.164933,0.455088,-0.453230,543 days,0.301095,-0.027552,0.978029,1043.0,0.110630,0.008657,0.042335,-0.079483,0.016308,0.388962,-0.364991,-0.001947,-0.693383,-0.176926,0.261356,0.132018,0.169646,0.234305,0.194816,-0.213334,-0.085410,-0.085929,-0.223035,-0.185946
5,out_sample,2023-10-01,2024-03-31,0.50,2,6.412281,4.343482,38.271538,0.485075,-0.167547,46 days,0.519126,3.147942,0.001922,499.0,0.273771,0.747803,0.249714,0.583737,1.138996,1.166870,2.661903,1.814170,3.303557,3.375407,0.203438,0.219253,0.108534,0.134964,0.248662,-0.047227,-0.119019,-0.034213,-0.038193,-0.046614
6,in_sample,2022-04-01,2023-09-30,0.55,2,-0.097338,-0.184626,-0.201887,0.482503,-0.482143,543 days,0.299270,-0.075661,0.939716,1042.0,0.107885,0.007602,0.046149,-0.088246,0.006668,0.370616,-0.344822,0.035378,-0.663609,-0.236837,0.269628,0.141177,0.184232,0.258422,0.200289,-0.221876,-0.093833,-0.094265,-0.245486,-0.199675
7,out_sample,2023-10-01,2024-03-31,0.55,2,6.977381,4.271527,39.165265,0.515147,-0.178152,48 days,0.519126,3.092902,0.002294,494.0,0.289048,0.734381,0.257679,0.603136,1.277086,1.186108,2.566635,1.763381,3.323094,3.348049,0.213004,0.224529,0.116072,0.138194,0.273705,-0.047227,-0.125139,-0.038109,-0.040867,-0.051496
8,in_sample,2022-04-01,2023-09-30,0.60,2,-0.117362,-0.207682,-0.231006,0.507960,-0.508049,543 days,0.302920,-0.111688,0.911112,1038.0,0.107388,0.004414,0.047546,-0.094977,-0.002273,0.364279,-0.345533,0.051122,-0.631067,-0.288512,0.276621,0.150442,0.197786,0.280663,0.206066,-0.225825,-0.103669,-0.102555,-0.265977,-0.213097
9,out_sample,2023-10-01,2024-03-31,0.60,2,7.314869,4.216353,39.446661,0.535582,-0.185437,48 days,0.519126,3.051391,0.002619,502.0,0.313237,0.735873,0.256001,0.608096,1.349062,1.242157,2.533092,1.683157,3.316270,3.328442,0.221690,0.228189,0.121068,0.139525,0.286950,-0.047227,-0.128738,-0.040544,-0.042289,-0.055150


In [337]:
df_performance_target_volatility_micro_1.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Micro_Grid_Performance-2022-04-01-2024-04-01.pickle')

In [339]:
%%time
df_performance_target_volatility_micro_2 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2022-10-01', end_date='2024-10-01',ticker_list=ticker_list)

Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
Run Dates: 
Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
{'target_volatility': 0.4, 'stop_loss_atr': 2}
0.4 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio

In [341]:
df_performance_target_volatility_micro_2.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Micro_Grid_Performance-2022-10-01-2024-10-01.pickle')

In [343]:
%%time
df_performance_target_volatility_micro_3 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2023-04-01', end_date='2025-04-01',ticker_list=ticker_list)

Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
Run Dates: 
Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
{'target_volatility': 0.4, 'stop_loss_atr': 2}
0.4 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio

In [ ]:
df_performance_target_volatility_micro_3.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Micro_Grid_Performance-2023-04-01-2025-04-01.pickle')

In [345]:
%%time
df_performance_target_volatility_micro_4 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2023-10-01', end_date='2025-10-01',ticker_list=ticker_list)

Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-09-30
end_date_os > last_available_date
Run Dates: 
Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-07-31
{'target_volatility': 0.4, 'stop_loss_atr': 2}
0.4 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash 

In [346]:
df_performance_target_volatility_micro_4.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Micro_Grid_Performance-2023-10-01-2025-10-01.pickle')

In [349]:
df_performance_target_volatility_micro = pd.concat([df_performance_target_volatility_micro_1, df_performance_target_volatility_micro_2,
                                                    df_performance_target_volatility_micro_3, df_performance_target_volatility_micro_4], axis=0, ignore_index=True)

In [353]:
df_performance_target_volatility_micro.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_Micro_Grid_Performance-2022-04-01-2025-10-01.pickle')

In [355]:
df_performance_target_volatility_micro['vol_tracking_error'] = (np.abs(df_performance_target_volatility_micro['annualized_std_dev'] - df_performance_target_volatility_micro['target_volatility'])/
                                                                df_performance_target_volatility_micro['target_volatility'])

In [357]:
in_sample_cond = (df_performance_target_volatility_micro.sampling_category == 'in_sample')
df_performance_target_volatility_micro_is = df_performance_target_volatility_micro[in_sample_cond].reset_index(drop=True)
df_performance_target_volatility_micro_os = df_performance_target_volatility_micro[~in_sample_cond].reset_index(drop=True)

In [359]:
df_performance_target_volatility_micro

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown,vol_tracking_error
0,in_sample,2022-04-01,2023-09-30,0.40,2,-0.042103,-0.120859,-0.105261,0.397764,-0.399985,543 days,0.299270,0.035298,0.971855,1048.0,0.104272,0.011220,0.033482,-0.065273,0.033853,0.376830,-0.419962,-0.110533,-0.789520,-0.060096,0.237468,0.111013,0.139584,1.877395e-01,0.184989,-0.194533,-0.067019,-0.069717,-0.181683,-0.158282,0.005591
1,out_sample,2023-10-01,2024-03-31,0.40,2,5.062797,4.528530,37.507222,0.414667,-0.134982,46 days,0.519126,3.291619,0.001196,494.0,0.245218,0.739358,0.211575,0.515382,0.862709,1.151671,2.894772,1.797564,3.220527,3.346189,0.180418,0.198319,0.089608,1.234330e-01,0.200878,-0.045149,-0.095449,-0.027456,-0.030829,-0.037460,0.036666
2,in_sample,2022-04-01,2023-09-30,0.45,2,-0.050502,-0.111321,-0.119184,0.429003,-0.423735,543 days,0.297445,0.033085,0.973619,1039.0,0.116199,0.011541,0.037903,-0.071947,0.025523,0.422577,-0.368814,-0.051284,-0.733602,-0.116706,0.252917,0.122545,0.154722,2.110710e-01,0.189765,-0.204613,-0.076348,-0.078182,-0.202557,-0.172095,0.046659
3,out_sample,2023-10-01,2024-03-31,0.45,2,5.824276,4.459586,38.589493,0.450482,-0.150929,46 days,0.519126,3.235811,0.001441,493.0,0.258946,0.763862,0.231678,0.552705,1.002997,1.158821,2.810420,1.815455,3.276705,3.369353,0.191620,0.210601,0.099153,1.294335e-01,0.225442,-0.046210,-0.106850,-0.030961,-0.034753,-0.041879,0.001071
4,in_sample,2022-04-01,2023-09-30,0.50,2,-0.074752,-0.152718,-0.164933,0.455088,-0.453230,543 days,0.301095,-0.027552,0.978029,1043.0,0.110630,0.008657,0.042335,-0.079483,0.016308,0.388962,-0.364991,-0.001947,-0.693383,-0.176926,0.261356,0.132018,0.169646,2.343046e-01,0.194816,-0.213334,-0.085410,-0.085929,-0.223035,-0.185946,0.089823
5,out_sample,2023-10-01,2024-03-31,0.50,2,6.412281,4.343482,38.271538,0.485075,-0.167547,46 days,0.519126,3.147942,0.001922,499.0,0.273771,0.747803,0.249714,0.583737,1.138996,1.166870,2.661903,1.814170,3.303557,3.375407,0.203438,0.219253,0.108534,1.349638e-01,0.248662,-0.047227,-0.119019,-0.034213,-0.038193,-0.046614,0.029851
6,in_sample,2022-04-01,2023-09-30,0.55,2,-0.097338,-0.184626,-0.201887,0.482503,-0.482143,543 days,0.299270,-0.075661,0.939716,1042.0,0.107885,0.007602,0.046149,-0.088246,0.006668,0.370616,-0.344822,0.035378,-0.663609,-0.236837,0.269628,0.141177,0.184232,2.584219e-01,0.200289,-0.221876,-0.093833,-0.094265,-0.245486,-0.199675,0.122721
7,out_sample,2023-10-01,2024-03-31,0.55,2,6.977381,4.271527,39.165265,0.515147,-0.178152,48 days,0.519126,3.092902,0.002294,494.0,0.289048,0.734381,0.257679,0.603136,1.277086,1.186108,2.566635,1.763381,3.323094,3.348049,0.213004,0.224529,0.116072,1.381940e-01,0.273705,-0.047227,-0.125139,-0.038109,-0.040867,-0.051496,0.063368
8,in_sample,2022-04-01,2023-09-30,0.60,2,-0.117362,-0.207682,-0.231006,0.507960,-0.508049,543 days,0.302920,-0.111688,0.911112,1038.0,0.107388,0.004414,0.047546,-0.094977,-0.002273,0.364279,-0.345533,0.051122,-0.631067,-0.288512,0.276621,0.150442,0.197786,2.806627e-01,0.206066,-0.225825,-0.103669,-0.102555,-0.265977,-0.213097,0.153401
9,out_sample,2023-10-01,2024-03-31,0.60,2,7.314869,4.216353,39.446661,0.535582,-0.185437,48 days,0.519126,3.051391,0.002619,502.0,0.313237,0.735873,0.256001,0.608096,1.349062,1.242157,2.533092,1.683157,3.316270,3.328442,0.221690,0.2

In [363]:
df_performance_target_volatility_micro_os#.groupby(['start_date']).size()

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown,vol_tracking_error
0,out_sample,2023-10-01,2024-03-31,0.40,2,5.062797,4.528530,37.507222,0.414667,-0.134982,46 days,0.519126,3.291619,0.001196,494.0,0.245218,0.739358,0.211575,0.515382,0.862709,1.151671,2.894772,1.797564,3.220527,3.346189,0.180418,0.198319,0.089608,1.234330e-01,0.200878,-0.045149,-0.095449,-0.027456,-0.030829,-0.037460,0.036666
1,out_sample,2023-10-01,2024-03-31,0.45,2,5.824276,4.459586,38.589493,0.450482,-0.150929,46 days,0.519126,3.235811,0.001441,493.0,0.258946,0.763862,0.231678,0.552705,1.002997,1.158821,2.810420,1.815455,3.276705,3.369353,0.191620,0.210601,0.099153,1.294335e-01,0.225442,-0.046210,-0.106850,-0.030961,-0.034753,-0.041879,0.001071
2,out_sample,2023-10-01,2024-03-31,0.50,2,6.412281,4.343482,38.271538,0.485075,-0.167547,46 days,0.519126,3.147942,0.001922,499.0,0.273771,0.747803,0.249714,0.583737,1.138996,1.166870,2.661903,1.814170,3.303557,3.375407,0.203438,0.219253,0.108534,1.349638e-01,0.248662,-0.047227,-0.119019,-0.034213,-0.038193,-0.046614,0.029851
3,out_sample,2023-10-01,2024-03-31,0.55,2,6.977381,4.271527,39.165265,0.515147,-0.178152,48 days,0.519126,3.092902,0.002294,494.0,0.289048,0.734381,0.257679,0.603136,1.277086,1.186108,2.566635,1.763381,3.323094,3.348049,0.213004,0.224529,0.116072,1.381940e-01,0.273705,-0.047227,-0.125139,-0.038109,-0.040867,-0.051496,0.063368
4,out_sample,2023-10-01,2024-03-31,0.60,2,7.314869,4.216353,39.446661,0.535582,-0.185437,48 days,0.519126,3.051391,0.002619,502.0,0.313237,0.735873,0.256001,0.608096,1.349062,1.242157,2.533092,1.683157,3.316270,3.328442,0.221690,0.228189,0.121068,1.395254e-01,0.286950,-0.047227,-0.128738,-0.040544,-0.042289,-0.055150,0.107363
5,out_sample,2023-10-01,2024-03-31,0.65,2,7.535787,4.171656,39.466385,0.549779,-0.190942,48 days,0.519126,3.018067,0.002909,504.0,0.326295,0.734196,0.252891,0.608361,1.410569,1.255963,2.501129,1.614270,3.298179,3.319919,0.229385,0.230920,0.124819,1.403816e-01,0.297353,-0.047227,-0.131051,-0.042107,-0.044207,-0.056793,0.154186
6,out_sample,2023-10-01,2024-03-31,0.70,2,7.614356,4.118735,39.204763,0.560591,-0.194220,48 days,0.519126,2.979364,0.003283,503.0,0.333317,0.724234,0.246967,0.603370,1.463034,1.251189,2.461010,1.533928,3.266199,3.296488,0.236120,0.232226,0.128229,1.407643e-01,0.307736,-0.047629,-0.132322,-0.043780,-0.046078,-0.059450,0.199155
7,out_sample,2024-04-01,2024-09-30,0.40,2,-0.110678,-1.075253,-1.050781,0.370891,-0.105329,174 days,0.087432,-0.522636,0.601863,49.0,-0.068132,-0.040015,-0.006745,0.007847,0.065160,-1.181713,-2.963698,-12.536550,-4.032203,0.237984,0.271761,0.102060,0.018790,6.641723e-02,0.350304,-0.072190,-0.029590,-0.003587,-0.002990,-0.022572,0.072771
8,out_sample,2024-04-01,2024-09-30,0.45,2,-0.124961,-1.036947,-1.058811,0.417684,-0.118020,174 days,0.087432,-0.522240,0.602138,49.0,-0.077389,-0.044968,-0.007588,0.008825,0.073246,-1.130342,-2.783620,-11.313339,-3.498052,0.315002,0.305720,0.114817,0.021138,7.471907e-02,0.394079,-0.081147,-0.033252,-0.004035,-0.003364,-0.025393,0.071813
9,out_sample,2024-04-01,2024-09-30,0.50,2,-0.139316,-1.006168,-1.066668,0.464584,-0.130609,174 days,0.087432,-0.521834,0.602421,49.0,-0.086794,-0.049909,-0.008430,0.009802,0.081314,-1.090172,-2.639575,-10.334806,-3.070726,0.376654,0.339682,0.127573,0.0

In [365]:
df_performance_target_volatility_micro_is.groupby(['target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)

annualized_sharpe_ratio                      \
                                   median      mean       std   
target_volatility                                               
0.40                             1.818428  1.408951  1.033023   
0.45                             1.799051  1.397682  1.019965   
0.50                             1.752078  1.349157  1.013800   
0.55                             1.742953  1.325141  1.016352   
0.60                             1.730304  1.301222  1.013957   
0.65                             1.723027  1.281294  1.013302   
0.70                             1.703517  1.247243  1.015121   

                  annualized_return                     max_drawdown  \
                             median      mean       std       median   
target_volatility                                                      
0.40                       0.707046  0.626071  0.488626    -0.224760   
0.45                       0.768161  0.678208  0.531959    -0.246122   
0.50                       0.806364  0.701299  0.561194    -0.267500   
0.55                       0.855210  0.729475  0.590560    -0.284786   
0.60                       0.884099  0.742813  0.609224    -0.295831   
0.65                       0.905154  0.750068  0.622976    -0.303524   
0.70                       0.909165  0.740905  0.628350    -0.311830   

                                      annualized_std_dev                      \
                       mean       std             median      mean       std   
target_volatility                                                              
0.40              -0.255329  0.100634           0.363065  0.358073  0.049840   
0.45              -0.276904  0.101946           0.394748  0.387966  0.051166   
0.50              -0.300038  0.105901           0.423187  0.415100  0.051111   
0.55              -0.318714  0.113288           0.450029  0.439708  0.053336   
0.60              -0.331443  0.123430           0.470527  0.459595  0.057944   
0.65              -0.340456  0.133075           0.483910  0.474946  0.063497   
0.70              -0.349894  0.144244           0.496350  0.487050  0.067813   

                  vol_tracking_error                     trade_count           \
                              median      mean       std      median     mean   
target_volatility                                                               
0.40                        0.093044  0.108320  0.120540      1069.5  1099.75   
0.45                        0.122782  0.137852  0.113703      1066.5  1098.00   
0.50                        0.153626  0.169801  0.102222      1067.0  1097.00   
0.55                        0.181766  0.200530  0.096975      1070.0  1098.75   
0.60                        0.215789  0.234009  0.096574      1069.5  1103.00   
0.65                        0.255523  0.269314  0.097687      1069.0  1104.75   
0.70                        0.290928  0.304215  0.096875      1063.0  1101.00   

                              BTC-USD_annualized_sharpe_ratio            \
                          std                          median      mean   
target_volatility                                                         
0.40               122.355697                        0.402808  0.585591   
0.45               123.954293                        0.441088  0.617039   
0.50               119.498954                        0.439909  0.608247   
0.55               122.363870                        0.453268  0.624454   
0.60               126.039676                        0.470145  0.650179   
0.65               128.621862                        0.478718  0.662633   
0.70               129.524772                        0.486189  0.668028   

                            ETH-USD_annualized_sharpe_ratio            \
                        std                          median      mean   
target_volatility                                                       
0.40               0.651611                        1.148339  0.8

In [367]:
df_performance_target_volatility_micro_os.groupby(['target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)

annualized_sharpe_ratio                      \
                                   median      mean       std   
target_volatility                                               
0.60                             1.242429  1.463302  2.198085   
0.65                             1.235831  1.455610  2.159014   
0.55                             1.237036  1.449872  2.264686   
0.70                             1.230914  1.437955  2.130764   
0.45                             1.148097  1.429708  2.419239   
0.50                             1.185975  1.427316  2.339894   
0.40                             1.099577  1.413108  2.472449   

                  annualized_return                     max_drawdown  \
                             median      mean       std       median   
target_volatility                                                      
0.60                       0.429331  2.008158  3.556371    -0.164375   
0.65                       0.432322  2.064799  3.664700    -0.166147   
0.55                       0.418685  1.916974  3.393230    -0.159656   
0.70                       0.435442  2.084660  3.703339    -0.167232   
0.45                       0.374415  1.612036  2.831547    -0.134475   
0.50                       0.391968  1.764225  3.120185    -0.149078   
0.40                       0.353577  1.414818  2.456948    -0.120155   

                                      annualized_std_dev                      \
                       mean       std             median      mean       std   
target_volatility                                                              
0.60              -0.153012  0.040926           0.503752  0.463684  0.108844   
0.65              -0.155274  0.042828           0.517635  0.475658  0.114166   
0.55              -0.148832  0.038046           0.478094  0.443086  0.099613   
0.70              -0.156636  0.043754           0.526472  0.484081  0.118008   
0.45              -0.134275  0.032476           0.399743  0.384497  0.070168   
0.50              -0.143375  0.036431           0.440558  0.415346  0.085097   
0.40              -0.123257  0.026469           0.356578  0.352430  0.055654   

                  vol_tracking_error                     trade_count          \
                              median      mean       std      median    mean   
target_volatility                                                              
0.60                        0.160414  0.227193  0.181407       313.0  294.00   
0.65                        0.203638  0.268218  0.175639       313.0  294.25   
0.55                        0.130739  0.194389  0.181114       310.5  290.75   
0.70                        0.247897  0.308456  0.168583       308.5  291.50   
0.45                        0.111682  0.146098  0.155259       314.0  292.50   
0.50                        0.118885  0.169308  0.170193       308.5  291.25   
0.40                        0.108555  0.137257  0.114440       315.0  293.25   

                              BTC-USD_annualized_sharpe_ratio            \
                          std                          median      mean   
target_volatility                                                         
0.60               198.284308                        1.130718  0.628241   
0.65               199.673692                        1.133329  0.636995   
0.55               194.643906                        1.108872  0.599098   
0.70               198.246480                        1.134827  0.640485   
0.45               195.102537                        0.994829  0.520723   
0.50               195.322596                        1.044262  0.555692   
0.40               194.958756                        0.952385  0.477428   

                            ETH-USD_annualized_sharpe_ratio            \
                        std                          median      mean   
target_volatility                                                       
0.60               1.083327                        0.882104  0.459261   
0

## Walk Forward Analysis with Target Volatility, ATR Multiple and ATR Window

In [374]:
import itertools

def generate_target_volatility_stop_loss_atr_params():
    parameter_grid = {
        "target_volatility": [0.5, 0.55, 0.60],
        "stop_loss_atr": np.arange(0.75, 3.1, 0.25).tolist(),
        "atr_window": [10, 14, 20, 30]
    }
    keys, values = zip(*parameter_grid.items())
    for prod in itertools.product(*values):
        yield dict(zip(keys, prod))

In [380]:
def run_walk_forward_target_volatility_stop_loss_atr(start_date, end_date, ticker_list):

    start_date = pd.Timestamp(start_date).date()
    end_date = pd.Timestamp(end_date).date()
    perf_cols = ['sampling_category', 'start_date', 'end_date', 'target_volatility', 'stop_loss_atr', 'atr_window', 'annualized_return', 'annualized_sharpe_ratio', 'calmar_ratio',
                 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate', 't_statistic', 'p_value', 'trade_count']
    ticker_perf_cols = ['annualized_return', 'annualized_sharpe_ratio', 'annualized_std_dev', 'max_drawdown']
    perf_cols.extend([f'{ticker}_{col}' for col in ticker_perf_cols for ticker in ticker_list])
    
    df_performance = pd.DataFrame(columns=perf_cols)
    
    IS_LEN = pd.DateOffset(months=18)
    OS_LEN = pd.DateOffset(months=6)
    start_date_is = start_date
    last_available_date = pd.Timestamp('2025-07-31').date()
    WARMUP_DAYS = 323
    while True:
        end_date_is = (start_date_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_date_os = (end_date_is + pd.Timedelta(days=1))
        end_date_os = (start_date_os + OS_LEN - pd.Timedelta(days=1)).date()
        fmt = "%Y-%m-%d"
        
        fields = [
            ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
            ("IS start",          start_date_is),
            ("IS end",            end_date_is),
            ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
            ("OS start",          start_date_os),
            ("OS end",            end_date_os),
        ]
        
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        ## Break Condition for While loop
        if end_date_os > end_date - pd.Timedelta(days=1):
            break

        if end_date_os > last_available_date:
            print('end_date_os > last_available_date')
            end_date_os = last_available_date
            fields = [
                ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
                ("IS start",          start_date_is),
                ("IS end",            end_date_is),
                ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
                ("OS start",          start_date_os),
                ("OS end",            end_date_os),
            ]
        
        print("Run Dates: ")
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
    
        for params in generate_target_volatility_stop_loss_atr_params():
            print(params)
            target_volatility = params['target_volatility']
            stop_loss_atr = params['stop_loss_atr']
            atr_window = params['atr_window']
            
            print(target_volatility, stop_loss_atr, atr_window)

            ## In Sample Dataframe
            print('Pulling In Sample Data!!')
            df_is = tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol(
                start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, weighted_signal_ewm_window=weighted_signal_ewm_window,
                rolling_r2_window=rolling_r2_window, lower_r_sqr_limit=lower_r_sqr_limit, upper_r_sqr_limit=upper_r_sqr_limit, r2_smooth_window=r2_smooth_window, r2_confirm_days=r2_confirm_days,
                log_std_window=log_std_window, coef_of_variation_window=coef_of_variation_window, vol_of_vol_z_score_window=vol_of_vol_z_score_window, vol_of_vol_p_min=vol_of_vol_p_min, r2_strong_threshold=r2_strong_threshold,
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                moving_avg_type=moving_avg_type, long_only=long_only, price_or_returns_calc=price_or_returns_calc,
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=atr_window, atr_multiplier=stop_loss_atr,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate,
                notional_threshold_pct=notional_threshold_pct, cooldown_counter_threshold=cooldown_counter_threshold,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date, 
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage, annualized_target_volatility=target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=use_specific_start_date, signal_start_date=start_date_is)
            df_is = df_is[df_is.index >= pd.Timestamp(start_date_is).date()]
            
            print('Calculating In Sample Asset Returns!!')
            df_is = perf.calculate_asset_level_returns(df_is, end_date_is, ticker_list)

            ## In Sample Performance Metrics
            print('Getting In Sample Performance Metrics!!')
            row_parameters_is = {
                'sampling_category': 'in_sample',
                'start_date': start_date_is,
                'end_date': end_date_is,
                'target_volatility': target_volatility,
                'stop_loss_atr': stop_loss_atr,
                'atr_window': atr_window
            }
            portfolio_perf_metrics_is = calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

            print('Getting In Sample Asset Performance!!')
            for ticker in ticker_list:
                ## In Sample
                ticker_perf_metrics_is = perf.calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_is = {key: ticker_perf_metrics_is[key] for key in ticker_perf_cols}
                ticker_perf_metrics_is = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_is.items()}
                portfolio_perf_metrics_is.update(ticker_perf_metrics_is)
            
            row_parameters_is.update(portfolio_perf_metrics_is)

            ## Assign in sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_is

        # Get Moving Average and Donchian Channel Weights with best performing in-sample Sharpe Ratio
        # in_sample_cond = (df_performance['sampling_category'] == 'in_sample')
        # date_cond = (df_performance['start_date'] == start_date_is)# & (df_performance['end_date'] == end_date_is)
        # walk_forward_run_cond = in_sample_cond & date_cond

        # ## Get Performing Parameters in the In-Sample period based on the Rolling R2 score
        # best_target_volatility = df_performance[walk_forward_run_cond].sort_values('annualized_sharpe_ratio', ascending=False)['target_volatility'].iloc[0]
        # best_stop_loss_atr = df_performance[walk_forward_run_cond].sort_values('annualized_sharpe_ratio', ascending=False)['stop_loss_atr'].iloc[0]
        
            ## Out of Sample Dataframe
            print('Pulling Out of Sample Data!!')
            df_os = tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol(
                start_date=start_date_os - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_os, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, weighted_signal_ewm_window=weighted_signal_ewm_window,
                rolling_r2_window=rolling_r2_window, lower_r_sqr_limit=lower_r_sqr_limit, upper_r_sqr_limit=upper_r_sqr_limit, r2_smooth_window=r2_smooth_window, r2_confirm_days=r2_confirm_days,
                log_std_window=log_std_window, coef_of_variation_window=coef_of_variation_window, vol_of_vol_z_score_window=vol_of_vol_z_score_window, vol_of_vol_p_min=vol_of_vol_p_min, r2_strong_threshold=r2_strong_threshold,
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                moving_avg_type=moving_avg_type, long_only=long_only, price_or_returns_calc=price_or_returns_calc,
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=atr_window, atr_multiplier=stop_loss_atr,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate,
                notional_threshold_pct=notional_threshold_pct, cooldown_counter_threshold=cooldown_counter_threshold,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date, 
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage, annualized_target_volatility=target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=use_specific_start_date, signal_start_date=start_date_os)
            df_os = df_os[df_os.index >= pd.Timestamp(start_date_os).date()]
                
            print('Calculating Out of Sample Asset Returns!!')
            df_os = perf.calculate_asset_level_returns(df_os, end_date_os, ticker_list)
            
            ## Out of Sample Performance Metrics
            print('Pulling Out of Sample Performance Metrics!!')
            row_parameters_os = {
                'sampling_category': 'out_sample',
                'start_date': start_date_os,
                'end_date': end_date_os,
                'target_volatility': target_volatility,
                'stop_loss_atr': stop_loss_atr,
                'atr_window': atr_window
            }
            portfolio_perf_metrics_os = calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)
    
            print('Getting Out of Sample Asset Performance!!')
            for ticker in ticker_list:
                ## Out of Sample
                ticker_perf_metrics_os = perf.calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_os = {key: ticker_perf_metrics_os[key] for key in ticker_perf_cols}
                ticker_perf_metrics_os = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_os.items()}
                portfolio_perf_metrics_os.update(ticker_perf_metrics_os)
            
            row_parameters_os.update(portfolio_perf_metrics_os)
    
            ## Assign out of sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_os

        start_date_is = (start_date_is + OS_LEN).date()
        
    return df_performance

In [382]:
%%time
df_performance_target_volatility_atr_1 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2022-04-01', end_date='2024-04-01', ticker_list=ticker_list)

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
{'target_volatility': 0.5, 'stop_loss_atr': 0.75, 'atr_window': 10}
0.5 0.75 10
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Managem

In [383]:
df_performance_target_volatility_atr_1

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,atr_window,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,0.50,0.75,10,-0.066283,-0.236276,-0.212762,0.387396,-0.311535,543 days,0.262774,-0.093594,0.925466,929.0,0.012490,0.017236,0.018116,-0.132386,0.114295,-0.138558,-0.311505,-0.208470,-1.182955,0.541804,0.244300,0.133282,0.183738,0.242209,0.204577,-0.247111,-0.052428,-0.089644,-0.212616,-0.068072
1,out_sample,2023-10-01,2024-03-31,0.50,0.75,10,5.053793,4.300050,38.686008,0.451932,-0.130636,29 days,0.491803,3.125208,0.002068,465.0,0.181997,0.821369,0.211108,0.448616,1.155948,0.923029,2.969633,1.540041,2.708286,3.436269,0.172667,0.227229,0.113374,0.141491,0.266768,-0.044881,-0.096323,-0.028672,-0.033794,-0.027041
2,in_sample,2022-04-01,2023-09-30,0.50,0.75,14,-0.058972,-0.213411,-0.191943,0.385835,-0.307236,543 days,0.264599,-0.064538,0.948565,933.0,0.009042,0.017051,0.028281,-0.132386,0.104937,-0.158904,-0.313554,-0.124500,-1.182955,0.474005,0.243160,0.134107,0.180300,0.243343,0.202559,-0.247107,-0.052679,-0.089644,-0.212616,-0.068072
3,out_sample,2023-10-01,2024-03-31,0.50,0.75,14,4.724893,4.149749,36.796320,0.455532,-0.128407,29 days,0.480874,3.018362,0.002906,460.0,0.182399,0.790583,0.189107,0.442124,1.155961,0.926085,2.886034,1.402708,2.669766,3.436305,0.172533,0.229437,0.111277,0.140078,0.266767,-0.044681,-0.096323,-0.038062,-0.033792,-0.027043
4,in_sample,2022-04-01,2023-09-30,0.50,0.75,20,-0.115104,-0.415073,-0.358527,0.387675,-0.321047,543 days,0.253650,-0.311881,0.755250,933.0,0.006537,0.016756,0.018347,-0.156827,0.100319,-0.174128,-0.316947,-0.206525,-1.388363,0.440055,0.246056,0.134341,0.180091,0.239307,0.205513,-0.247107,-0.053085,-0.089605,-0.243251,-0.068072
5,out_sample,2023-10-01,2024-03-31,0.50,0.75,20,4.355553,4.009851,34.683066,0.453792,-0.125582,31 days,0.475410,2.919650,0.003947,456.0,0.182360,0.779064,0.164639,0.443427,1.056770,0.925840,2.854691,1.192436,2.677258,3.223158,0.172534,0.231946,0.111005,0.138604,0.269916,-0.044681,-0.096323,-0.040386,-0.033790,-0.029973
6,in_sample,2022-04-01,2023-09-30,0.50,0.75,30,-0.117275,-0.426018,-0.354383,0.381978,-0.330928,543 days,0.255474,-0.324523,0.745667,942.0,0.006456,0.016541,0.022456,-0.156867,0.088801,-0.174660,-0.323571,-0.170684,-1.388624,0.350246,0.246025,0.133794,0.179718,0.235523,0.210657,-0.247103,-0.052997,-0.085929,-0.243306,-0.082348
7,out_sample,2023-10-01,2024-03-31,0.50,0.75,30,4.853205,4.252796,39.808542,0.448851,-0.121914,29 days,0.469945,3.092423,0.002298,458.0,0.177246,0.794503,0.183162,0.443056,1.113227,0.891893,2.902063,1.376603,2.681091,3.338613,0.170986,0.231720,0.110141,0.139744,0.267328,-0.044681,-0.096323,-0.032900,-0.033940,-0.029950
8,in_sample,2022-04-01,2023-09-30,0.50,1.00,10,-0.020740,-0.059301,-0.060971,0.404792,-0.340163,543 days,0.273723,0.112914,0.910140,969.0,0.016433,0.017199,0.059770,-0.089441,0.092477,-0.111068,-0.311949,0.135018,-0.821532,0.375451,0.243277,0.130880,0.176289,0.238126,0.201454,-0.247103,-0.052494,-0.085929,-0.212616,-0.092984
9,out_sample,2023-10-01,2024-03-31,0.50,1.00,10,4.973963,4.052043,30.889564,0.468222,-0.161024,48 days,0.508197,2.945034,0.003651,477.0,0.082152,0.716124,0.223095,0.611845,1.150538,0.270670,2.580094,1.604850,3.468268,3.410297,0.192045,0.232425,0.111254,0.138947,0.253138,-0.079568,-0.119155,-0.034382,-0.029

In [384]:
df_performance_target_volatility_atr_1.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_ATR_Multiple_Window_Performance-2022-04-01-2024-04-01.pickle')

In [385]:
%%time
df_performance_target_volatility_atr_2 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2022-10-01', end_date='2024-10-01',ticker_list=ticker_list)

Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
Run Dates: 
Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
{'target_volatility': 0.5, 'stop_loss_atr': 0.75, 'atr_window': 10}
0.5 0.75 10
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Managem

In [386]:
df_performance_target_volatility_atr_2.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_ATR_Multiple_Window_Performance-2022-10-01-2024-10-01.pickle')

In [387]:
%%time
df_performance_target_volatility_atr_3 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2023-04-01', end_date='2025-04-01',ticker_list=ticker_list)

Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
Run Dates: 
Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
{'target_volatility': 0.5, 'stop_loss_atr': 0.75, 'atr_window': 10}
0.5 0.75 10
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Managem

In [388]:
df_performance_target_volatility_atr_3.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_ATR_Multiple_Window_Performance-2023-04-01-2025-04-01.pickle')

In [389]:
%%time
df_performance_target_volatility_atr_4 = run_walk_forward_target_volatility_stop_loss_atr(start_date='2023-10-01', end_date='2025-10-01',ticker_list=ticker_list)

Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-09-30
end_date_os > last_available_date
Run Dates: 
Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-07-31
{'target_volatility': 0.5, 'stop_loss_atr': 0.75, 'atr_window': 10}
0.5 0.75 10
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targe

In [390]:
df_performance_target_volatility_atr_4.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_ATR_Multiple_Window_Performance-2023-10-01-2025-10-01.pickle')

In [406]:
df_performance_target_volatility_atr = pd.concat([df_performance_target_volatility_atr_1, df_performance_target_volatility_atr_2,
                                                  df_performance_target_volatility_atr_3, df_performance_target_volatility_atr_4], axis=0, ignore_index=True)

In [408]:
df_performance_target_volatility_atr.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Target_Volatility_ATR_Multiple_Window_Performance-2022-04-01-2025-10-01.pickle')

In [412]:
df_performance_target_volatility_atr['vol_tracking_error'] = (np.abs(df_performance_target_volatility_atr['annualized_std_dev'] - df_performance_target_volatility_atr['target_volatility'])/
                                                              df_performance_target_volatility_atr['target_volatility'])

In [414]:
in_sample_cond = (df_performance_target_volatility_atr.sampling_category == 'in_sample')
df_performance_target_volatility_atr_is = df_performance_target_volatility_atr[in_sample_cond].reset_index(drop=True)
df_performance_target_volatility_atr_os = df_performance_target_volatility_atr[~in_sample_cond].reset_index(drop=True)

In [416]:
df_performance_target_volatility_atr_is.groupby(['target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)

annualized_sharpe_ratio                      \
                                   median      mean       std   
target_volatility                                               
0.50                             1.710188  1.327508  0.888443   
0.55                             1.708092  1.307195  0.888639   
0.60                             1.690375  1.286586  0.883953   

                  annualized_return                     max_drawdown  \
                             median      mean       std       median   
target_volatility                                                      
0.50                       0.816676  0.674772  0.476276    -0.255338   
0.55                       0.858787  0.703470  0.501315    -0.271425   
0.60                       0.886586  0.717233  0.516887    -0.274934   

                                      annualized_std_dev                      \
                       mean       std             median      mean       std   
target_volatility                                                              
0.50              -0.284467  0.087515           0.426628  0.418699  0.036940   
0.55              -0.301948  0.093044           0.451997  0.443731  0.039031   
0.60              -0.313372  0.100517           0.471574  0.463224  0.041585   

                  vol_tracking_error                     trade_count  \
                              median      mean       std      median   
target_volatility                                                      
0.50                        0.146744  0.162602  0.073880      1090.0   
0.55                        0.178188  0.193216  0.070965      1102.0   
0.60                        0.214044  0.227961  0.069308      1104.0   

                                          BTC-USD_annualized_sharpe_ratio  \
                         mean         std                          median   
target_volatility                                                           
0.50               1079.39375  186.369927                        0.389149   
0.55               1082.45000  188.217818                        0.407864   
0.60               1085.23125  189.614114                        0.399656   

                                      ETH-USD_annualized_sharpe_ratio  \
                       mean       std                          median   
target_volatility                                                       
0.50               0.531127  0.606650                        1.054565   
0.55               0.546364  0.608889                        1.026137   
0.60               0.571071  0.607777                        1.009145   

                                      SOL-USD_annualized_sharpe_ratio  \
                       mean       std                          median   
target_volatility                                                       
0.50               0.720138  0.664647                        0.273608   
0.55               0.702491  0.637535                        0.271033   
0.60               0.691492  0.625846                        0.233966   

                                      ADA-USD_annualized_sharpe_ratio  \
                       mean       std                          median   
target_volatility                                                       
0.50               0.300030  0.213527                        0.911384   
0.55               0.319003  0.204577                        0.878316   
0.60               0.312512  0.197780                        0.847548   

                                      AVAX-USD_annualized_sharpe_ratio  \
                       mean       std                           median   
target_volatility                                                        
0.50               0.631874  0.891673                         1.474878   
0.55               0.635172  0.881371                         1.495447   
0.60               0.626944  0.865553                         1.501675   

                                       
               

In [418]:
df_performance_target_volatility_atr_os.groupby(['target_volatility']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)

annualized_sharpe_ratio                      \
                                   median      mean       std   
target_volatility                                               
0.60                             0.850124  1.369919  1.939335   
0.55                             0.839729  1.358360  1.995568   
0.50                             0.780954  1.331766  2.053936   

                  annualized_return                     max_drawdown  \
                             median      mean       std       median   
target_volatility                                                      
0.60                       0.281704  1.880561  2.948013    -0.153941   
0.55                       0.269438  1.803909  2.827479    -0.148721   
0.50                       0.238696  1.655351  2.594199    -0.141008   

                                      annualized_std_dev                      \
                       mean       std             median      mean       std   
target_volatility                                                              
0.60              -0.152547  0.030920           0.472512  0.454259  0.091350   
0.55              -0.148176  0.028603           0.449049  0.435156  0.083979   
0.50              -0.142398  0.027281           0.418557  0.408090  0.071952   

                  vol_tracking_error                     trade_count  \
                              median      mean       std      median   
target_volatility                                                      
0.60                        0.212480  0.242901  0.152250       291.0   
0.55                        0.183548  0.208808  0.152689       287.5   
0.50                        0.162885  0.183819  0.143903       286.5   

                                         BTC-USD_annualized_sharpe_ratio  \
                        mean         std                          median   
target_volatility                                                          
0.60               288.83125  170.129112                        1.037372   
0.55               286.55000  168.181349                        1.043078   
0.50               285.54375  167.092933                        0.928135   

                                      ETH-USD_annualized_sharpe_ratio  \
                       mean       std                          median   
target_volatility                                                       
0.60               0.554753  0.966029                        0.940839   
0.55               0.529619  0.980705                        0.943150   
0.50               0.483562  0.980327                        0.904211   

                                      SOL-USD_annualized_sharpe_ratio  \
                       mean       std                          median   
target_volatility                                                       
0.60               0.445186  2.179441                        0.321149   
0.55               0.455720  2.250893                        0.232317   
0.50               0.441853  2.360245                        0.140158   

                                      ADA-USD_annualized_sharpe_ratio       \
                       mean       std                          median mean   
target_volatility                                                            
0.60              -1.673441  4.215012                       -1.626197 -inf   
0.55              -1.862658  4.492747                       -1.784497 -inf   
0.50              -2.096268  4.826060                       -1.998372 -inf   

                      AVAX-USD_annualized_sharpe_ratio                       
                  std                           median      mean        std  
target_volatility                                                            
0.60              NaN                         0.457559 -4.568310  10.354131  
0.55              NaN                         0.370474 -5.100737  11.195170  
0.50              NaN                         0.289478 -5.729333  12.210029

In [456]:
in_sample_vol_tracking_cond = (df_performance_target_volatility_atr_is.vol_tracking_error <= 0.20)
out_sample_vol_tracking_cond = (df_performance_target_volatility_atr_os.vol_tracking_error <= 0.20)
df_target_vol_stop_loss_is = df_performance_target_volatility_atr_is.groupby(['target_volatility','stop_loss_atr']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)
in_sample_vol_tracking_cond = (df_target_vol_stop_loss_is[('vol_tracking_error', 'mean')] <= 0.20)
df_target_vol_stop_loss_os[in_sample_vol_tracking_cond]

annualized_sharpe_ratio                      \
                                                 median      mean       std   
target_volatility stop_loss_atr                                               
0.60              2.00                         1.244071  1.464139  1.966342   
                  1.75                         1.247811  1.463913  1.971807   
0.50              2.25                         1.219808  1.454518  2.117531   
0.55              1.75                         1.242743  1.450999  2.031232   
                  2.00                         1.238820  1.450712  2.025925   
0.50              1.75                         1.191462  1.428440  2.098318   
                  2.00                         1.187812  1.428118  2.093180   
0.60              1.50                         1.252093  1.426183  1.937476   
0.55              1.50                         1.246861  1.416906  2.002400   
0.50              1.50                         1.190716  1.387703  2.064598   
0.55              1.00                         0.917132  1.205755  1.943721   
                  1.25                         0.895041  1.202230  1.959589   
0.50              1.25                         0.833153  1.170765  2.016837   
                  1.00                         0.850394  1.170601  1.991929   
                  0.75                         0.773493  0.887003  2.325698   

                                annualized_return                      \
                                           median      mean       std   
target_volatility stop_loss_atr                                         
0.60              2.00                   0.430063  2.009579  3.182961   
                  1.75                   0.431719  2.013534  3.192988   
0.50              2.25                   0.404589  1.841740  2.921700   
0.55              1.75                   0.421083  1.922343  3.046446   
                  2.00                   0.419432  1.918301  3.036908   
0.50              1.75                   0.394150  1.768750  2.800303   
                  2.00                   0.392666  1.765368  2.792411   
0.60              1.50                   0.433549  1.864255  2.942740   
0.55              1.50                   0.422788  1.799600  2.842009   
0.50              1.50                   0.394105  1.647786  2.599054   
0.55              1.00                   0.302979  1.535897  2.495078   
                  1.25                   0.287209  1.456358  2.318566   
0.50              1.25                   0.259665  1.345823  2.147828   
                  1.00                   0.269531  1.406262  2.295658   
                  0.75                   0.234041  1.242654  2.103474   

                                max_drawdown                      \
                                      median      mean       std   
target_volatility stop_loss_atr                                    
0.60              2.00             -0.164111 -0.152940  0.036544   
                  1.75             -0.163516 -0.153384  0.036945   
0.50              2.25             -0.149078 -0.142296  0.031533   
0.55              1.75             -0.158775 -0.149119  0.034303   
                  2.00             -0.159382 -0.148763  0.033967   
0.50              1.75             -0.149078 -0.143609  0.032855   
                  2.00             -0.149078 -0.143312  0.032515   
0.60              1.50             -0.162858 -0.154101  0.037709   
0.55              1.50             -0.158149 -0.149477  0.034698   
0.50              1.50             -0.149078 -0.143900  0.033092   
0.55              1.00             -0.146409 -0.149027  0.009998   
                  1.25             -0.161753 -0.156102  0.026057   
0.50              1.25             -0.153182 -0.148852  0.023476   
                  1.00             -0.137143 -0.142442  0.011488   
                  0.75             -0.132243 -0.136534  0.012846   

                                annualized_std_dev                   

In [458]:
df_target_vol_stop_loss_os = df_performance_target_volatility_atr_os.groupby(['target_volatility','stop_loss_atr']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)
out_sample_vol_tracking_cond = (df_target_vol_stop_loss_os[('vol_tracking_error', 'mean')] <= 0.20)
df_target_vol_stop_loss_os[out_sample_vol_tracking_cond]

annualized_sharpe_ratio                      \
                                                 median      mean       std   
target_volatility stop_loss_atr                                               
0.55              3.00                         1.271938  1.488446  2.047109   
                  2.75                         1.271938  1.488446  2.047109   
                  2.50                         1.271938  1.488429  2.047083   
                  2.25                         1.272685  1.479064  2.052552   
0.50              2.75                         1.219171  1.463510  2.111992   
                  3.00                         1.219171  1.463510  2.111992   
                  2.50                         1.219171  1.463494  2.111967   
                  2.25                         1.219808  1.454518  2.117531   
0.55              1.75                         1.242743  1.450999  2.031232   
                  2.00                         1.238820  1.450712  2.025925   
0.50              1.75                         1.191462  1.428440  2.098318   
                  2.00                         1.187812  1.428118  2.093180   
0.55              1.50                         1.246861  1.416906  2.002400   
0.50              1.50                         1.190716  1.387703  2.064598   
                  1.25                         0.833153  1.170765  2.016837   

                                annualized_return                      \
                                           median      mean       std   
target_volatility stop_loss_atr                                         
0.55              3.00                   0.433409  2.012615  3.187420   
                  2.75                   0.433409  2.012615  3.187420   
                  2.50                   0.433409  2.012560  3.187323   
                  2.25                   0.433716  2.008702  3.190007   
0.50              2.75                   0.404331  1.845056  2.919457   
                  3.00                   0.404331  1.845056  2.919457   
                  2.50                   0.404331  1.845010  2.919374   
                  2.25                   0.404589  1.841740  2.921700   
0.55              1.75                   0.421083  1.922343  3.046446   
                  2.00                   0.419432  1.918301  3.036908   
0.50              1.75                   0.394150  1.768750  2.800303   
                  2.00                   0.392666  1.765368  2.792411   
0.55              1.50                   0.422788  1.799600  2.842009   
0.50              1.50                   0.394105  1.647786  2.599054   
                  1.25                   0.259665  1.345823  2.147828   

                                max_drawdown                      \
                                      median      mean       std   
target_volatility stop_loss_atr                                    
0.55              3.00             -0.154274 -0.146141  0.031866   
                  2.75             -0.154274 -0.146141  0.031866   
                  2.50             -0.154274 -0.146141  0.031866   
                  2.25             -0.154163 -0.147591  0.033103   
0.50              2.75             -0.149078 -0.141010  0.030126   
                  3.00             -0.149078 -0.141010  0.030126   
                  2.50             -0.149078 -0.141010  0.030126   
                  2.25             -0.149078 -0.142296  0.031533   
0.55              1.75             -0.158775 -0.149119  0.034303   
                  2.00             -0.159382 -0.148763  0.033967   
0.50              1.75             -0.149078 -0.143609  0.032855   
                  2.00             -0.149078 -0.143312  0.032515   
0.55              1.50             -0.158149 -0.149477  0.034698   
0.50              1.50             -0.149078 -0.143900  0.033092   
                  1.25             -0.153182 -0.148852  0.023476   

                                annualized_std_dev                   

In [462]:
df_target_vol_stop_loss_atr_window_is = df_performance_target_volatility_atr_is.groupby(['target_volatility','stop_loss_atr','atr_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)
in_sample_vol_tracking_cond = (df_target_vol_stop_loss_atr_window_is[('vol_tracking_error', 'mean')] <= 0.20)
df_target_vol_stop_loss_atr_window_is[in_sample_vol_tracking_cond]

annualized_sharpe_ratio            \
                                                            median      mean   
target_volatility stop_loss_atr atr_window                                     
0.50              1.75          20                        1.759908  1.397208   
                                30                        1.754025  1.396559   
                  1.00          14                        1.744092  1.394914   
0.55              1.00          14                        1.740084  1.383953   
                  1.75          20                        1.750446  1.374636   
                                30                        1.744712  1.374509   
0.50              2.25          14                        1.795195  1.370183   
                                10                        1.795633  1.369724   
                  1.00          20                        1.702117  1.367863   
0.55              1.00          20                        1.701847  1.359437   
0.60              1.75          30                        1.731598  1.352660   
                                20                        1.737305  1.352349   
0.50              1.00          10                        1.683971  1.351066   
                  2.00          20                        1.752078  1.349157   
0.55              2.25          14                        1.785278  1.346466   
                                10                        1.785743  1.345915   
0.50              2.00          14                        1.753055  1.345734   
                                10                        1.754316  1.341787   
                                30                        1.752954  1.340312   
0.55              1.00          10                        1.678590  1.338653   
0.50              1.50          14                        1.736489  1.331953   
                                20                        1.716802  1.328738   
0.55              2.00          14                        1.743962  1.321428   
0.60              2.25          10                        1.769637  1.320570   
0.55              2.00          10                        1.745248  1.317210   
0.50              1.75          14                        1.760068  1.314727   
0.55              1.50          14                        1.731322  1.314252   
                                20                        1.711262  1.311679   
0.50              1.25          30                        1.534514  1.303251   
                  1.75          10                        1.765675  1.300413   
0.60              2.00          14                        1.731344  1.297221   
0.50              0.75          10                        1.714263  1.293089   
0.60              1.50          14                        1.717712  1.292932   
                  2.00          10                        1.732600  1.292685   
0.55              1.75          14                        1.750607  1.291008   
                  1.25          30                        1.529046  1.290078   
0.60              1.50          20                        1.697803  1.289746   
0.50              1.50          30                        1.648545  1.287099   
0.55              1.75          10                        1.756409  1.278662   
0.60              1.25          30                        1.515507  1.274259   
0.50              1.00          30                        1.708469  1.272409   
0.55              1.50          30                        1.640274  1.269102   
0.50              0.75          14                        1.660650  1.267828   
0.60              1.75          14                        1.737459  1.267332   
0.50              1.25          10                        1.623786  1.259836   
0.55              1.00          30                        1.708740  1.258450   
0.50              1.50          10                        1.739443  1.257955   
0.60              1.75          10         

In [464]:
df_target_vol_stop_loss_atr_window_os = df_performance_target_volatility_atr_os.groupby(['target_volatility','stop_loss_atr','atr_window']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','mean'), ascending=False)
out_sample_vol_tracking_cond = (df_target_vol_stop_loss_atr_window_os[('vol_tracking_error', 'mean')] <= 0.20)
df_target_vol_stop_loss_atr_window_os[out_sample_vol_tracking_cond]

annualized_sharpe_ratio            \
                                                            median      mean   
target_volatility stop_loss_atr atr_window                                     
0.55              2.25          10                        1.272685  1.488820   
                  3.00          30                        1.271938  1.488446   
                  2.25          14                        1.271938  1.488446   
                  2.50          10                        1.271938  1.488446   
                                14                        1.271938  1.488446   
                                20                        1.271938  1.488446   
                  2.75          10                        1.271938  1.488446   
                                14                        1.271938  1.488446   
                                20                        1.271938  1.488446   
                                30                        1.271938  1.488446   
                  3.00          10                        1.271938  1.488446   
                                14                        1.271938  1.488446   
                                20                        1.271938  1.488446   
                  2.50          30                        1.271938  1.488378   
                  2.25          30                        1.234970  1.470029   
                                20                        1.232966  1.468960   
0.50              2.25          10                        1.219808  1.463829   
                  2.50          10                        1.219171  1.463510   
                  2.25          14                        1.219171  1.463510   
                  2.50          20                        1.219171  1.463510   
                  3.00          30                        1.219171  1.463510   
                                20                        1.219171  1.463510   
                                14                        1.219171  1.463510   
                                10                        1.219171  1.463510   
                  2.50          14                        1.219171  1.463510   
                  2.75          20                        1.219171  1.463510   
                                14                        1.219171  1.463510   
                                10                        1.219171  1.463510   
                                30                        1.219171  1.463510   
                  2.50          30                        1.219171  1.463444   
0.55              1.75          10                        1.242743  1.456823   
                  2.00          10                        1.238294  1.451412   
                                30                        1.238820  1.451187   
                                14                        1.237550  1.450377   
                                20                        1.237036  1.449872   
                  1.75          14                        1.229459  1.449530   
                                20                        1.229373  1.449509   
                                30                        1.231300  1.448135   
0.50              2.25          30                        1.183848  1.445914   
                                20                        1.181787  1.444818   
0.55              1.50          14                        1.246861  1.439903   
0.50              1.75          10                        1.191462  1.434051   
                  2.00          10                        1.187091  1.428761   
                                30                        1.187812  1.428647   
0.55              1.50          20                        1.240379  1.428626   
0.50              2.00          14                        1.186356  1.427748   
                                20                        1.185975  1.427316   
                  1.75          14         

In [430]:
df_performance_target_volatility_atr_os[(df_performance_target_volatility_atr_os.target_volatility == 0.60) & (df_performance_target_volatility_atr_os.stop_loss_atr > 2)]

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,atr_window,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown,vol_tracking_error
104,out_sample,2023-10-01,2024-03-31,0.6,2.25,10,7.643715,4.287663,41.220017,0.535995,-0.185437,48 days,0.524590,3.101830,0.002230,503.0,0.346646,0.734609,0.255770,0.608092,1.349061,1.368209,2.529607,1.681738,3.316255,3.328438,0.219858,0.228187,0.121049,1.395252e-01,0.284018,-0.047227,-0.128738,-0.040544,-0.042289,-0.055154,0.106675
105,out_sample,2023-10-01,2024-03-31,0.6,2.25,14,7.643715,4.287663,41.220017,0.535995,-0.185437,48 days,0.524590,3.101830,0.002230,503.0,0.346646,0.734609,0.255770,0.608092,1.349061,1.368209,2.529607,1.681738,3.316255,3.328438,0.219858,0.228187,0.121049,1.395252e-01,0.284018,-0.047227,-0.128738,-0.040544,-0.042289,-0.055154,0.106675
106,out_sample,2023-10-01,2024-03-31,0.6,2.25,20,7.643715,4.287663,41.220017,0.535995,-0.185437,48 days,0.524590,3.101830,0.002230,503.0,0.346646,0.734609,0.255770,0.608092,1.349061,1.368209,2.529607,1.681738,3.316255,3.328438,0.219858,0.228187,0.121049,1.395252e-01,0.284018,-0.047227,-0.128738,-0.040544,-0.042289,-0.055154,0.106675
107,out_sample,2023-10-01,2024-03-31,0.6,2.25,30,7.644632,4.287945,41.224963,0.535984,-0.185437,48 days,0.524590,3.102031,0.002228,501.0,0.346646,0.734609,0.255770,0.608092,1.349025,1.368208,2.529607,1.681737,3.316255,3.328375,0.219858,0.228187,0.121049,1.395252e-01,0.286950,-0.047227,-0.128738,-0.040544,-0.042289,-0.055154,0.106694
108,out_sample,2023-10-01,2024-03-31,0.6,2.50,10,7.643715,4.287663,41.220017,0.535995,-0.185437,48 days,0.524590,3.101830,0.002230,503.0,0.346646,0.734609,0.255770,0.608092,1.349061,1.368209,2.529607,1.681738,3.316255,3.328438,0.219858,0.228187,0.121049,1.395252e-01,0.284018,-0.047227,-0.128738,-0.040544,-0.042289,-0.055154,0.106675
109,out_sample,2023-10-01,2024-03-31,0.6,2.50,14,7.643715,4.287663,41.220017,0.535995,-0.185437,48 days,0.524590,3.101830,0.002230,503.0,0.346646,0.734609,0.255770,0.608092,1.349061,1.368209,2.529607,1.681738,3.316255,3.328438,0.219858,0.228187,0.121049,1.395252e-01,0.284018,-0.047227,-0.128738,-0.040544,-0.042289,-0.055154,0.106675
110,out_sample,2023-10-01,2024-03-31,0.6,2.50,20,7.643715,4.287663,41.220017,0.535995,-0.185437,48 days,0.524590,3.101830,0.002230,503.0,0.346646,0.734609,0.255770,0.608092,1.349061,1.368209,2.529607,1.681738,3.316255,3.328438,0.219858,0.228187,0.121049,1.395252e-01,0.284018,-0.047227,-0.128738,-0.040544,-0.042289,-0.055154,0.106675
111,out_sample,2023-10-01,2024-03-31,0.6,2.50,30,7.642736,4.287382,41.214742,0.536004,-0.185437,48 days,0.524590,3.101629,0.002231,501.0,0.346646,0.734609,0.255770,0.608092,1.349025,1.368209,2.529607,1.681738,3.316255,3.328375,0.219858,0.228187,0.121049,1.395252e-01,0.286950,-0.047227,-0.128738,-0.040544,-0.042289,-0.055154,0.106660
112,out_sample,2023-10-01,2024-03-31,0.6,2.75,10,7.643715,4.287663,41.220017,0.535995,-0.185437,48 days,0.524590,3.101830,0.002230,503.0,0.346646,0.734609,0.255770,0.608092,1.349061,1.368209,2.529607,1.681738,3.316255,3.328438,0.219858,0.228187,0.121049,1.395252e-01,0.284018,-0.047227,-0.128738,-0.040544,-0.042289,-0.055154,0.106675
113,out_sample,2023-10-01,2024-03-31,0.6,2.75,14,7.643715,4.287663,41.220017,0.535995,-0.185437,48 days,0.524590,3.101830,0.002230,503.0,0.346646,0.734609,0.255770,0.608092,1.349061,1.368209,2.529607,

## Walk Forward Analysis for Stop Loss Cooldown Sweep

In [471]:
import itertools

def generate_stop_loss_cooldown_params():
    parameter_grid = {
        "target_volatility": [0.50, 0.55],
        "stop_loss_atr": [2.25, 2.50, 2.75],
        "atr_window": [20],
        "stop_loss_cooldown": [0, 1, 2, 3, 5, 7, 10]
    }
    keys, values = zip(*parameter_grid.items())
    for prod in itertools.product(*values):
        yield dict(zip(keys, prod))

In [475]:
def run_walk_forward_stop_loss_cooldown(start_date, end_date, ticker_list):

    start_date = pd.Timestamp(start_date).date()
    end_date = pd.Timestamp(end_date).date()
    perf_cols = ['sampling_category', 'start_date', 'end_date', 'target_volatility', 'stop_loss_atr', 'atr_window', 'stop_loss_cooldown', 'annualized_return', 'annualized_sharpe_ratio', 'calmar_ratio',
                 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate', 't_statistic', 'p_value', 'trade_count']
    ticker_perf_cols = ['annualized_return', 'annualized_sharpe_ratio', 'annualized_std_dev', 'max_drawdown']
    perf_cols.extend([f'{ticker}_{col}' for col in ticker_perf_cols for ticker in ticker_list])
    
    df_performance = pd.DataFrame(columns=perf_cols)
    
    IS_LEN = pd.DateOffset(months=18)
    OS_LEN = pd.DateOffset(months=6)
    start_date_is = start_date
    last_available_date = pd.Timestamp('2025-07-31').date()
    WARMUP_DAYS = 323
    while True:
        end_date_is = (start_date_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_date_os = (end_date_is + pd.Timedelta(days=1))
        end_date_os = (start_date_os + OS_LEN - pd.Timedelta(days=1)).date()
        fmt = "%Y-%m-%d"
        
        fields = [
            ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
            ("IS start",          start_date_is),
            ("IS end",            end_date_is),
            ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
            ("OS start",          start_date_os),
            ("OS end",            end_date_os),
        ]
        
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        ## Break Condition for While loop
        if end_date_os > end_date - pd.Timedelta(days=1):
            break

        if end_date_os > last_available_date:
            print('end_date_os > last_available_date')
            end_date_os = last_available_date
            fields = [
                ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
                ("IS start",          start_date_is),
                ("IS end",            end_date_is),
                ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
                ("OS start",          start_date_os),
                ("OS end",            end_date_os),
            ]
        
        print("Run Dates: ")
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
    
        for params in generate_stop_loss_cooldown_params():
            print(params)
            target_volatility = params['target_volatility']
            stop_loss_atr = params['stop_loss_atr']
            atr_window = params['atr_window']
            stop_loss_cooldown = params['stop_loss_cooldown']
            
            print(target_volatility, stop_loss_atr, atr_window, stop_loss_cooldown)

            ## In Sample Dataframe
            print('Pulling In Sample Data!!')
            df_is = tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol(
                start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, weighted_signal_ewm_window=weighted_signal_ewm_window,
                rolling_r2_window=rolling_r2_window, lower_r_sqr_limit=lower_r_sqr_limit, upper_r_sqr_limit=upper_r_sqr_limit, r2_smooth_window=r2_smooth_window, r2_confirm_days=r2_confirm_days,
                log_std_window=log_std_window, coef_of_variation_window=coef_of_variation_window, vol_of_vol_z_score_window=vol_of_vol_z_score_window, vol_of_vol_p_min=vol_of_vol_p_min, r2_strong_threshold=r2_strong_threshold,
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                moving_avg_type=moving_avg_type, long_only=long_only, price_or_returns_calc=price_or_returns_calc,
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=atr_window, atr_multiplier=stop_loss_atr,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate,
                notional_threshold_pct=notional_threshold_pct, cooldown_counter_threshold=stop_loss_cooldown,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date, 
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage, annualized_target_volatility=target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=use_specific_start_date, signal_start_date=start_date_is)
            df_is = df_is[df_is.index >= pd.Timestamp(start_date_is).date()]
            
            print('Calculating In Sample Asset Returns!!')
            df_is = perf.calculate_asset_level_returns(df_is, end_date_is, ticker_list)

            ## In Sample Performance Metrics
            print('Getting In Sample Performance Metrics!!')
            row_parameters_is = {
                'sampling_category': 'in_sample',
                'start_date': start_date_is,
                'end_date': end_date_is,
                'target_volatility': target_volatility,
                'stop_loss_atr': stop_loss_atr,
                'atr_window': atr_window,
                'stop_loss_cooldown': stop_loss_cooldown
            }
            portfolio_perf_metrics_is = calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

            print('Getting In Sample Asset Performance!!')
            for ticker in ticker_list:
                ## In Sample
                ticker_perf_metrics_is = perf.calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_is = {key: ticker_perf_metrics_is[key] for key in ticker_perf_cols}
                ticker_perf_metrics_is = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_is.items()}
                portfolio_perf_metrics_is.update(ticker_perf_metrics_is)
            
            row_parameters_is.update(portfolio_perf_metrics_is)

            ## Assign in sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_is

        # Get Moving Average and Donchian Channel Weights with best performing in-sample Sharpe Ratio
        # in_sample_cond = (df_performance['sampling_category'] == 'in_sample')
        # date_cond = (df_performance['start_date'] == start_date_is)# & (df_performance['end_date'] == end_date_is)
        # walk_forward_run_cond = in_sample_cond & date_cond

        # ## Get Performing Parameters in the In-Sample period based on the Rolling R2 score
        # best_target_volatility = df_performance[walk_forward_run_cond].sort_values('annualized_sharpe_ratio', ascending=False)['target_volatility'].iloc[0]
        # best_stop_loss_atr = df_performance[walk_forward_run_cond].sort_values('annualized_sharpe_ratio', ascending=False)['stop_loss_atr'].iloc[0]
        
            ## Out of Sample Dataframe
            print('Pulling Out of Sample Data!!')
            df_os = tf.apply_target_volatility_position_sizing_continuous_strategy_with_rolling_r_sqr_vol_of_vol(
                start_date=start_date_os - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_os, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, weighted_signal_ewm_window=weighted_signal_ewm_window,
                rolling_r2_window=rolling_r2_window, lower_r_sqr_limit=lower_r_sqr_limit, upper_r_sqr_limit=upper_r_sqr_limit, r2_smooth_window=r2_smooth_window, r2_confirm_days=r2_confirm_days,
                log_std_window=log_std_window, coef_of_variation_window=coef_of_variation_window, vol_of_vol_z_score_window=vol_of_vol_z_score_window, vol_of_vol_p_min=vol_of_vol_p_min, r2_strong_threshold=r2_strong_threshold,
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                moving_avg_type=moving_avg_type, long_only=long_only, price_or_returns_calc=price_or_returns_calc,
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=atr_window, atr_multiplier=stop_loss_atr,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate,
                notional_threshold_pct=notional_threshold_pct, cooldown_counter_threshold=stop_loss_cooldown,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date, 
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage, annualized_target_volatility=target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=use_specific_start_date, signal_start_date=start_date_os)
            df_os = df_os[df_os.index >= pd.Timestamp(start_date_os).date()]
                
            print('Calculating Out of Sample Asset Returns!!')
            df_os = perf.calculate_asset_level_returns(df_os, end_date_os, ticker_list)
            
            ## Out of Sample Performance Metrics
            print('Pulling Out of Sample Performance Metrics!!')
            row_parameters_os = {
                'sampling_category': 'out_sample',
                'start_date': start_date_os,
                'end_date': end_date_os,
                'target_volatility': target_volatility,
                'stop_loss_atr': stop_loss_atr,
                'atr_window': atr_window,
                'stop_loss_cooldown': stop_loss_cooldown
            }
            portfolio_perf_metrics_os = calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)
    
            print('Getting Out of Sample Asset Performance!!')
            for ticker in ticker_list:
                ## Out of Sample
                ticker_perf_metrics_os = perf.calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_os = {key: ticker_perf_metrics_os[key] for key in ticker_perf_cols}
                ticker_perf_metrics_os = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_os.items()}
                portfolio_perf_metrics_os.update(ticker_perf_metrics_os)
            
            row_parameters_os.update(portfolio_perf_metrics_os)
    
            ## Assign out of sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_os

        start_date_is = (start_date_is + OS_LEN).date()
        
    return df_performance

In [477]:
%%time
df_performance_stop_loss_cooldown_1 = run_walk_forward_stop_loss_cooldown(start_date='2022-04-01', end_date='2024-04-01', ticker_list=ticker_list)

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
{'target_volatility': 0.5, 'stop_loss_atr': 2.25, 'atr_window': 20, 'stop_loss_cooldown': 0}
0.5 2.25 20 0
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Pos

In [478]:
df_performance_stop_loss_cooldown_1

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,atr_window,stop_loss_cooldown,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,0.50,2.25,20,0,-0.122275,-0.283240,-0.251310,0.421900,-0.486550,543 days,0.297445,-0.189822,0.849519,1177.0,0.108225,0.001560,0.041851,-0.079525,-0.001270,0.376785,-0.434518,-0.005781,-0.693647,-0.303352,0.259932,0.131267,0.169044,0.197410,0.194561,-0.207031,-0.085228,-0.085929,-0.223035,-0.206995
1,out_sample,2023-10-01,2024-03-31,0.50,2.25,20,0,6.710878,4.421868,40.053704,0.485639,-0.167547,46 days,0.519126,3.203357,0.001604,497.0,0.305537,0.749629,0.249620,0.583783,1.138794,1.301459,2.667430,1.813649,3.303840,3.375113,0.201824,0.219227,0.108519,0.134962,0.246115,-0.047227,-0.119019,-0.034213,-0.038178,-0.046600
2,in_sample,2022-04-01,2023-09-30,0.50,2.25,20,1,-0.122275,-0.283240,-0.251310,0.421900,-0.486550,543 days,0.297445,-0.189822,0.849519,1177.0,0.108225,0.001560,0.041851,-0.079525,-0.001270,0.376785,-0.434518,-0.005781,-0.693647,-0.303352,0.259932,0.131267,0.169044,0.197410,0.194561,-0.207031,-0.085228,-0.085929,-0.223035,-0.206995
3,out_sample,2023-10-01,2024-03-31,0.50,2.25,20,1,6.710878,4.421868,40.053704,0.485639,-0.167547,46 days,0.519126,3.203357,0.001604,497.0,0.305537,0.749629,0.249620,0.583783,1.138794,1.301459,2.667430,1.813649,3.303840,3.375113,0.201824,0.219227,0.108519,0.134962,0.246115,-0.047227,-0.119019,-0.034213,-0.038178,-0.046600
4,in_sample,2022-04-01,2023-09-30,0.50,2.25,20,2,-0.103815,-0.230785,-0.217140,0.420338,-0.478101,543 days,0.299270,-0.124969,0.900594,1169.0,0.109577,0.010453,0.042228,-0.079488,0.007778,0.383949,-0.346796,-0.002797,-0.693411,-0.238236,0.260110,0.130881,0.169343,0.197409,0.194650,-0.216671,-0.085276,-0.085929,-0.223035,-0.196184
5,out_sample,2023-10-01,2024-03-31,0.50,2.25,20,2,6.710878,4.421868,40.053704,0.485639,-0.167547,46 days,0.519126,3.203357,0.001604,497.0,0.305537,0.749629,0.249620,0.583783,1.138794,1.301459,2.667430,1.813649,3.303840,3.375113,0.201824,0.219227,0.108519,0.134962,0.246115,-0.047227,-0.119019,-0.034213,-0.038178,-0.046600
6,in_sample,2022-04-01,2023-09-30,0.50,2.25,20,3,-0.091546,-0.195694,-0.196270,0.419753,-0.466428,543 days,0.301095,-0.081753,0.934873,1159.0,0.110585,0.008716,0.042335,-0.079483,0.016308,0.388748,-0.364389,-0.001947,-0.693382,-0.176926,0.261361,0.131354,0.169646,0.197408,0.194816,-0.213329,-0.085410,-0.085929,-0.223035,-0.185946
7,out_sample,2023-10-01,2024-03-31,0.50,2.25,20,3,6.710878,4.421868,40.053704,0.485639,-0.167547,46 days,0.519126,3.203357,0.001604,497.0,0.305537,0.749629,0.249620,0.583783,1.138794,1.301459,2.667430,1.813649,3.303840,3.375113,0.201824,0.219227,0.108519,0.134962,0.246115,-0.047227,-0.119019,-0.034213,-0.038178,-0.046600
8,in_sample,2022-04-01,2023-09-30,0.50,2.25,20,5,-0.090289,-0.192169,-0.193595,0.420561,-0.466380,543 days,0.299270,-0.077399,0.938335,1150.0,0.111477,0.010049,0.042335,-0.079483,0.015539,0.393073,-0.350825,-0.001947,-0.693380,-0.182568,0.263903,0.132230,0.169646,0.197408,0.196291,-0.212397,-0.085408,-0.085929,-0.223035,-0.186873
9,out_sample,2023-10-01,2024-03-31,0.50,2.25,20,5,6.710878,4.421868,40.053704,0.485639,-0.167547,46 days,0.519126,3.203357,0.001604,497.0,0.305537,0.749629,0.249620,0.583783,1.138794,1.301459,2.667430,1.813649,3.303840,3.375113,0.201824,0.219227,0.108519,0.1349

In [479]:
df_performance_stop_loss_cooldown_1.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Stop_Loss_Cooldown_Performance-2022-04-01-2024-04-01.pickle')

In [480]:
%%time
df_performance_stop_loss_cooldown_2 = run_walk_forward_stop_loss_cooldown(start_date='2022-10-01', end_date='2024-10-01',ticker_list=ticker_list)

Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
Run Dates: 
Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
{'target_volatility': 0.5, 'stop_loss_atr': 2.25, 'atr_window': 20, 'stop_loss_cooldown': 0}
0.5 2.25 20 0
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Pos

In [481]:
df_performance_stop_loss_cooldown_2.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Stop_Loss_Cooldown_Performance-2022-10-01-2024-10-01.pickle')

In [482]:
%%time
df_performance_stop_loss_cooldown_3 = run_walk_forward_stop_loss_cooldown(start_date='2023-04-01', end_date='2025-04-01',ticker_list=ticker_list)

Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
Run Dates: 
Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
{'target_volatility': 0.5, 'stop_loss_atr': 2.25, 'atr_window': 20, 'stop_loss_cooldown': 0}
0.5 2.25 20 0
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Pos

In [483]:
df_performance_stop_loss_cooldown_3.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Stop_Loss_Cooldown_Performance-2023-04-01-2025-04-01.pickle')

In [484]:
%%time
df_performance_stop_loss_cooldown_4 = run_walk_forward_stop_loss_cooldown(start_date='2023-10-01', end_date='2025-10-01',ticker_list=ticker_list)

Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-09-30
end_date_os > last_available_date
Run Dates: 
Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-07-31
{'target_volatility': 0.5, 'stop_loss_atr': 2.25, 'atr_window': 20, 'stop_loss_cooldown': 0}
0.5 2.25 20 0
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Pulling Out of Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
C

In [495]:
df_performance_stop_loss_cooldown_4.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Stop_Loss_Cooldown_Performance-2023-10-01-2025-10-01.pickle')

In [497]:
df_performance_stop_loss_cooldown = pd.concat([df_performance_stop_loss_cooldown_1, df_performance_stop_loss_cooldown_2,
                                               df_performance_stop_loss_cooldown_3, df_performance_stop_loss_cooldown_4], axis=0, ignore_index=True)

In [499]:
df_performance_stop_loss_cooldown.to_pickle(
    '/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Stop_Loss_Cooldown_Performance-2022-04-01-2025-10-01.pickle')

In [501]:
df_performance_stop_loss_cooldown['vol_tracking_error'] = (np.abs(df_performance_stop_loss_cooldown['annualized_std_dev'] - df_performance_stop_loss_cooldown['target_volatility'])/
                                                           df_performance_stop_loss_cooldown['target_volatility'])

In [503]:
in_sample_cond = (df_performance_stop_loss_cooldown.sampling_category == 'in_sample')
df_performance_stop_loss_cooldown_is = df_performance_stop_loss_cooldown[in_sample_cond].reset_index(drop=True)
df_performance_stop_loss_cooldown_os = df_performance_stop_loss_cooldown[~in_sample_cond].reset_index(drop=True)

In [507]:
df_performance_stop_loss_cooldown_is.groupby(['target_volatility','stop_loss_atr','atr_window','stop_loss_cooldown']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                               median   
target_volatility stop_loss_atr atr_window stop_loss_cooldown                           
0.50              2.25          20         10                                1.802372   
0.55              2.25          20         10                                1.791644   
0.50              2.25          20         5                                 1.790357   
                                           2                                 1.790296   
                                           7                                 1.789962   
                  2.75          20         5                                 1.787936   
                  2.50          20         10                                1.787936   
                  2.75          20         10                                1.787936   
                                           7                                 1.787936   
                                           3                                 1.787936   
                                           2                                 1.787936   
                                           1                                 1.787936   
                                           0                                 1.787936   
                  2.50          20         7                                 1.787936   
                                           5                                 1.787936   
                                           3                                 1.787936   
                                           2                                 1.787936   
                                           1                                 1.787936   
                                           0                                 1.787936   
                  2.25          20         3                                 1.783720   
0.55              2.25          20         2                                 1.780980   
                                           5                                 1.779229   
                                           7                                 1.778694   
                  2.50          20         2                                 1.778527   
                  2.75          20         0                                 1.778527   
                  2.50          20         10                                1.778527   
                                           7                                 1.778527   
                                           5                                 1.778527   
                                           3                                 1.778527   
                  2.75          20         10                                1.778527   
                  2.50          20         1                                 1.778527   
                                           0                                 1.778527   
                  2.75          20         3                                 1.778527   
                                           5                                 1.778527   
                                           7                                 1.778527   
                                           2                                 1.778527   
                                           1                                 1.778527   
0.50              2.25          20         0                                 1.777563   
                                           1                                 1.777563   
0.55              2.25          20         3                                 1.772943   
                                           1                                 1.768828   
                                           0                                 1.768828   

                                                        

In [517]:
locked_target_vol_config_cond = ((df_performance_stop_loss_cooldown_os.target_volatility == 0.55) &
                                 (df_performance_stop_loss_cooldown_os.stop_loss_atr == 2.50) &
                                 (df_performance_stop_loss_cooldown_os.atr_window == 20))
df_performance_stop_loss_cooldown_os[locked_target_vol_config_cond].groupby(['target_volatility','stop_loss_atr','atr_window','stop_loss_cooldown']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                               median   
target_volatility stop_loss_atr atr_window stop_loss_cooldown                           
0.55              2.5           20         0                                 1.271938   
                                           1                                 1.271938   
                                           2                                 1.271938   
                                           3                                 1.271938   
                                           5                                 1.271938   
                                           7                                 1.271938   
                                           10                                1.271938   

                                                                         \
                                                                   mean   
target_volatility stop_loss_atr atr_window stop_loss_cooldown             
0.55              2.5           20         0                   1.488446   
                                           1                   1.488446   
                                           2                   1.488446   
                                           3                   1.488446   
                                           5                   1.488446   
                                           7                   1.488446   
                                           10                  1.488446   

                                                                         \
                                                                    std   
target_volatility stop_loss_atr atr_window stop_loss_cooldown             
0.55              2.5           20         0                   2.288737   
                                           1                   2.288737   
                                           2                   2.288737   
                                           3                   2.288737   
                                           5                   2.288737   
                                           7                   2.288737   
                                           10                  2.288737   

                                                              annualized_return  \
                                                                         median   
target_volatility stop_loss_atr atr_window stop_loss_cooldown                     
0.55              2.5           20         0                           0.433409   
                                           1                           0.433409   
                                           2                           0.433409   
                                           3                           0.433409   
                                           5                           0.433409   
                                           7                           0.433409   
                                           10                          0.433409   

                                                                         \
                                                                   mean   
target_volatility stop_loss_atr atr_window stop_loss_cooldown             
0.55              2.5           20         0                   2.012615   
                                           1                   2.012615   
                                           2                   2.012615   
                                           3                   2.012615   
                                           5                   2.012615   
                                           7                   2.012615   
                                           10                  2.012615   

                                                                         \
            

In [515]:
locked_target_vol_config_cond = ((df_performance_stop_loss_cooldown_os.target_volatility == 0.55) &
                                 (df_performance_stop_loss_cooldown_os.stop_loss_atr == 2.25) &
                                 (df_performance_stop_loss_cooldown_os.atr_window == 20))
df_performance_stop_loss_cooldown_os[locked_target_vol_config_cond].groupby(['target_volatility','stop_loss_atr','atr_window','stop_loss_cooldown']).agg(agg_dict).sort_values(('annualized_sharpe_ratio','median'), ascending=False)

annualized_sharpe_ratio  \
                                                                               median   
target_volatility stop_loss_atr atr_window stop_loss_cooldown                           
0.55              2.25          20         10                                1.295070   
                                           0                                 1.269034   
                                           1                                 1.269034   
                                           2                                 1.253714   
                                           7                                 1.248808   
                                           5                                 1.245123   
                                           3                                 1.232966   

                                                                         \
                                                                   mean   
target_volatility stop_loss_atr atr_window stop_loss_cooldown             
0.55              2.25          20         10                  1.500012   
                                           0                   1.486994   
                                           1                   1.486994   
                                           2                   1.479334   
                                           7                   1.476881   
                                           5                   1.475039   
                                           3                   1.468960   

                                                                         \
                                                                    std   
target_volatility stop_loss_atr atr_window stop_loss_cooldown             
0.55              2.25          20         10                  2.281605   
                                           0                   2.289647   
                                           1                   2.289647   
                                           2                   2.294504   
                                           7                   2.296079   
                                           5                   2.297268   
                                           3                   2.301228   

                                                              annualized_return  \
                                                                         median   
target_volatility stop_loss_atr atr_window stop_loss_cooldown                     
0.55              2.25          20         10                          0.443110   
                                           0                           0.432169   
                                           1                           0.432169   
                                           2                           0.425641   
                                           7                           0.423582   
                                           5                           0.422040   
                                           3                           0.416982   

                                                                         \
                                                                   mean   
target_volatility stop_loss_atr atr_window stop_loss_cooldown             
0.55              2.25          20         10                  2.017465   
                                           0                   2.011995   
                                           1                   2.011995   
                                           2                   2.008731   
                                           7                   2.007702   
                                           5                   2.006931   
                                           3                   2.004401   

                                                                         \
            

In [513]:
df_performance_stop_loss_cooldown_os[((df_performance_stop_loss_cooldown_os.target_volatility == 0.55) &
                                     (df_performance_stop_loss_cooldown_os.stop_loss_atr == 2.25) &
                                     (df_performance_stop_loss_cooldown_os.atr_window == 20))]

,sampling_category,start_date,end_date,target_volatility,stop_loss_atr,atr_window,stop_loss_cooldown,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown,vol_tracking_error
21,out_sample,2023-10-01,2024-03-31,0.55,2.25,20,0,7.330498,4.356019,41.147372,0.516822,-0.178152,48 days,0.524590,3.152701,0.001892,499.0,0.324376,0.738649,0.257447,0.603180,1.277103,1.327529,2.579484,1.761918,3.323371,3.348084,0.211271,0.224452,0.116053,1.381912e-01,0.270909,-0.047227,-0.125139,-0.038110,-0.040852,-0.051496,0.060323
22,out_sample,2023-10-01,2024-03-31,0.55,2.25,20,1,7.330498,4.356019,41.147372,0.516822,-0.178152,48 days,0.524590,3.152701,0.001892,499.0,0.324376,0.738649,0.257447,0.603180,1.277103,1.327529,2.579484,1.761918,3.323371,3.348084,0.211271,0.224452,0.116053,1.381912e-01,0.270909,-0.047227,-0.125139,-0.038110,-0.040852,-0.051496,0.060323
23,out_sample,2023-10-01,2024-03-31,0.55,2.25,20,2,7.330498,4.356019,41.147372,0.516822,-0.178152,48 days,0.524590,3.152701,0.001892,499.0,0.324376,0.738649,0.257447,0.603180,1.277103,1.327529,2.579484,1.761918,3.323371,3.348084,0.211271,0.224452,0.116053,1.381912e-01,0.270909,-0.047227,-0.125139,-0.038110,-0.040852,-0.051496,0.060323
24,out_sample,2023-10-01,2024-03-31,0.55,2.25,20,3,7.330498,4.356019,41.147372,0.516822,-0.178152,48 days,0.524590,3.152701,0.001892,499.0,0.324376,0.738649,0.257447,0.603180,1.277103,1.327529,2.579484,1.761918,3.323371,3.348084,0.211271,0.224452,0.116053,1.381912e-01,0.270909,-0.047227,-0.125139,-0.038110,-0.040852,-0.051496,0.060323
25,out_sample,2023-10-01,2024-03-31,0.55,2.25,20,5,7.330498,4.356019,41.147372,0.516822,-0.178152,48 days,0.524590,3.152701,0.001892,499.0,0.324376,0.738649,0.257447,0.603180,1.277103,1.327529,2.579484,1.761918,3.323371,3.348084,0.211271,0.224452,0.116053,1.381912e-01,0.270909,-0.047227,-0.125139,-0.038110,-0.040852,-0.051496,0.060323
26,out_sample,2023-10-01,2024-03-31,0.55,2.25,20,7,7.330498,4.356019,41.147372,0.516822,-0.178152,48 days,0.524590,3.152701,0.001892,499.0,0.324376,0.738649,0.257447,0.603180,1.277103,1.327529,2.579484,1.761918,3.323371,3.348084,0.211271,0.224452,0.116053,1.381912e-01,0.270909,-0.047227,-0.125139,-0.038110,-0.040852,-0.051496,0.060323
27,out_sample,2023-10-01,2024-03-31,0.55,2.25,20,10,7.330498,4.356019,41.147372,0.516822,-0.178152,48 days,0.524590,3.152701,0.001892,499.0,0.324376,0.738649,0.257447,0.603180,1.277103,1.327529,2.579484,1.761918,3.323371,3.348084,0.211271,0.224452,0.116053,1.381912e-01,0.270909,-0.047227,-0.125139,-0.038110,-0.040852,-0.051496,0.060323
63,out_sample,2024-04-01,2024-09-30,0.55,2.25,20,0,-0.146856,-0.946110,-1.036972,0.509224,-0.141620,174 days,0.087432,-0.495937,0.620537,48.0,-0.094468,-0.052258,-0.009271,0.010779,0.089365,-1.046609,-2.537303,-9.534175,-2.721093,0.427129,0.372444,0.136021,0.025834,9.132256e-02,0.481623,-0.099007,-0.039240,-0.004932,-0.004110,-0.031036,0.074138
64,out_sample,2024-04-01,2024-09-30,0.55,2.25,20,1,-0.146856,-0.946110,-1.036972,0.509224,-0.141620,174 days,0.087432,-0.495937,0.620537,48.0,-0.094468,-0.052258,-0.009271,0.010779,0.089365,-1.046609,-2.537303,-9.534175,-2.721093,0.427129,0.372444,0.136021,0.025834,9.132256e-02,0.481623,-0.099007,-0.039240,-0.004932,-0.004110,-0.031036,0.074138
65,out_sample,2024-04-01,2024-09-30,0.55,2.25,20,2,-0.146856,-0.946110,-1.036972,0.509224,-0.141620,174 days,0.087432,-0.495937,0.620537,48.0,